In [127]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
from urllib.parse import urljoin
import xlsxwriter
from openpyxl import load_workbook
import re

In [ ]:
data = pd.read_csv(r'C:\Users\mariu\Downloads\liste_gemeinden_ger.csv', delimiter=';')
data = data.dropna(thresh=8)

In [ ]:
data_hessen= data[data['Land'] == 6]
data_scrape = data_hessen[['Gemeindename', 'plz']]

In [ ]:

def get_url1(name):
    e.send_keys(name)
    e.send_keys(Keys.ENTER)
    soup = BeautifulSoup(d.page_source)
    d.find_element_by_id('suchfeld').clear()
    time.sleep(0.7)
    links = soup.find('tbody').find_all('a') 
    return [link.get('href') for link in links if 'Votemanager' not in link]

In [ ]:
# set up webdriver and search field element
d=webdriver.Chrome("C:/Users/mariu/Downloads/chromedriver_win32 (2)/chromedriver.exe")
d.get('https://wahlen.votemanager.de/#')
e = d.find_element_by_id('suchfeld')

# search for each municipality and get the url to the municipality specific overview page
data_scrape['url'] = data_scrape['Gemeindename'].apply(get_url1)

In [ ]:
data_scrape = data_scrape[data_scrape.astype(str)['url']!='[]'].reset_index(drop=True)

# 2. look at municipalities were 2 results were found
data_scrape['results'] = data_scrape['url'].str.len()

data_scrape['url1'] = np.nan

for row in data_scrape.itertuples():
    if row[5] == 1:
        data_scrape.loc[row[0], 'url1'] = row[4]
    if row[5] == 2:
        data_scrape.loc[row[0], 'url1'] = row[4][0]
    if row[5] >= 3:
        data_scrape.loc[row[0], 'url1'] = 'Check again'
        
#data_scrape.drop(columns=['url', 'results'], inplace=True)

In [ ]:
data_scrape.to_csv(r'C:\Users\mariu\Downloads\scrape_step.csv')

In [ ]:
data_scrape['url']

In [ ]:
data_hessen = data_hessen.reset_index(drop=True)
for row in data_hessen.itertuples():
    data_hessen.loc[row[0], 'url1'] = [link for link in row[3] if str(row[2]) in link]

---

In [ ]:
data_scrape = pd.read_csv(r'C:\Users\mariu\Downloads\hessen_scrape.csv', delimiter=';')

In [ ]:
def get_url2(url1):
    """ To get the urls specific to the Ratswahl elections.
    It takes as input the overview url for each municipality from wahlen.regioit
    and returns a list of all Ratswahl-url from its html text.
    """
    base = url1
    r = requests.get(url1)
    soup = BeautifulSoup(r.text)
    
    kom_row  = [ r for r in soup.find_all('tr') if 'Kommunalwahlen' in r.text ]
    
    res = [link.get('href') for link in BeautifulSoup(str(kom_row)).find_all('a') if '27.03.2011' in link]
    
    if res:
        href=res.pop()
        return urljoin(base,href)
    else:
        return np.nan

In [ ]:
#dates_hessen = ['18.03.2001', '26.03.2006', '27.03.2011', '06.03.2016']
data_scrape['url_kom_11'] = data_scrape['url1'].apply(get_url2) # repeat this step 4 times

In [ ]:
def get_url3(url2):
    r = requests.get(url2) 
    soup = BeautifulSoup(r.text)
    base = url2
    hrefs = [link.get('href') for link in soup.find_all('a')]
    
    matches = ['Stadtverordnetenwahl_Hessen', 'Gemeindewahl_Hessen']
    
    href_2=[href for href in hrefs if any(x in href for x in matches)]
    
    if href_2:
        s = href_2.pop()
        return urljoin(base,s)
    else:
        return np.nan

In [ ]:
r = requests.get(data_url_16['url_kom_16'][2]) 
soup = BeautifulSoup(r.text)
base = data_url_16['url_kom_16'][2]
hrefs = [link.get('href') for link in soup.find_all('a')]
    
matches = ['Gemeindewahl_Hessen_', 'Stadtverordnetenwahl']
    
href_2=[href for href in hrefs if any(x in href for x in matches)]
href_2

In [ ]:
#data_url_01 = data_scrape[~data_scrape['url_kom_01'].isna()].reset_index(drop=True)
#data_url_06 = data_scrape[~data_scrape['url_kom_06'].isna()].reset_index(drop=True)
data_url_11 = data_scrape[~data_scrape['url_kom_11'].isna()].reset_index(drop=True)
#data_url_16 = data_scrape[~data_scrape['url_kom_16'].isna()].reset_index(drop=True)

#data_url_01['url3_01'] = data_url_01['url_kom_01'].apply(get_url3)
#data_url_06['url3_06'] = data_url_06['url_kom_06'].apply(get_url3)
data_url_11['url3_11'] = data_url_11['url_kom_11'].apply(get_url3)
#data_url_16['url3_16'] = data_url_16['url_kom_16'].apply(get_url3)

data_url_11.iloc[0,4]

In [ ]:
data_url_11.iloc[1,4]

In [ ]:
data_urls = [#data_url_01[['url3_01', 'Gemeindename', 'plz']],  data_url_06[['url3_6', 'Gemeindename', 'plz']],
             data_url_11[['Gemeindename', 'plz', 'url3_11']]]#, data_url_16[['Gemeindename', 'plz', 'url3_11']] ]

#for file in range(4):
#    data_hessen = pd.merge(data_hessen, data_urls[file], on=['Gemeindename', 'plz'], how='left', validate='1:1')
    
data_urls

In [ ]:
list_cities = data_url_01['Gemeindename'].unique()
iterations = len(data_url_01['url3_01'])
count = 0
data_url_01 = data_url_01[~data_url_01['url3_01'].isna()].reset_index(drop=True)
data_votes_2 = pd.DataFrame()
#data_2001 = pd.DataFrame()
#data_2006 = pd.DataFrame()
#data_2011 = pd.DataFrame()
#data_2016 = pd.DataFrame()

for n in range(len(data_url_01['url3_01'])):
    
    print(f'City {n}/{iterations}')
    
    r = requests.get(data_url_01['url3_01'][n])
    
    soup = BeautifulSoup(r.text)
    
    if soup.find_all(lambda tag: tag.name=='table') == []:
        print(f'{list_cities[n]} has no entry')
    
    else:
        table = soup.find_all(lambda tag: tag.name=='table')[-1]        
        rows_votes = table.find_all(lambda tag: tag.name=='tbody')
        
        if rows_votes == []:
            print(f'{list_cities[n]} has no entry')
            
        else:
            data_votes = pd.DataFrame(data=[rows_votes]).T.rename({0:'candidate'}, axis=1)
            data_votes['votes'] = data_votes['candidate'].astype(str).str.replace('.','').str.extract(r'([0-9]+[^%])')
                        
            data_votes['candidate'] = data_votes['candidate'].astype(str).str.replace('ß', 'ss').str.replace('Ü', 'U').str.replace('ü', 'ue').str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('.', '').str.replace('é', 'e')
            data_votes['candidates'] = data_votes['candidate'].str.extract(r'([A-Z][a-z]+[\s][A-Z][a-z]+[-]?[A-Z]*[a-z]*[,]*[\s]*[A-Z]*[a-z]*)')
            
            data_votes_2['votes'] = data_votes[data_votes["votes"].str.contains('<', na=False)].reset_index(drop=True)['votes'].str.replace('<', '')
            data_votes_2['candidates'] = data_votes['candidates'].dropna().reset_index(drop=True)
            data_votes_2['city'] = list_cities[n]
            data_votes_2['year'] = 2001
            data_2001 = data_2001.append(data_votes_2).dropna().reset_index(drop=True)
            
#             if len(data_votes) == len(data_candidates_2):
#                 data_hessen_2 = pd.concat([data_votes, data_candidates_2], axis=1)
#                 data_hessen_2['year'] = 2016
#                 data_hessen_2['city'] = data_url_16['Gemeindename'][n]
#                 data_hessen_2['plz'] = data_url_16['plz'][n]
#                 data_2011 = data_2011.append(data_hessen_2)
#                 count += 1
#                 print(f'{list_cities[n]} worked. {count}')
        
#             else:
#                 print(f'Please check {list_cities[n]} again')
        
#                 if len(data_votes) > len(data_candidates_2):
#                     print('some candidates are missing')
#                    [,]*[\s]*[A-Z]*[a-z]*
#                 elif len(data_votes) < len(data_candidates_2):
#                     print('too many candidates')
    

In [ ]:
list_cities = data_url_11['Gemeindename'].unique()
iterations = len(data_url_11['url3_11'])
count = 0
data_url_11 = data_url_11[~data_url_11['url3_11'].isna()].reset_index(drop=True)
data_votes_2 = pd.DataFrame()
#data_2006_incumbent = pd.DataFrame()
#data_2011_incumbent = pd.DataFrame()
data_2016_incumbent = pd.DataFrame()
#data_2021_incumbent = pd.DataFrame()

for n in range(len(data_url_11['url3_11'])):
    
    print(f'City {n}/{iterations}')
    
    r = requests.get(data_url_11['url3_11'][n])
    
    soup = BeautifulSoup(r.text)
    
    if soup.find_all(lambda tag: tag.name=='table') == []:
        print(f'{list_cities[n]} has no entry')
    
    else:
        table = soup.find_all(lambda tag: tag.name=='table')[0]        
        rows_votes = table.find_all(lambda tag: tag.name=='td')
        
        if rows_votes == []:
            print(f'{list_cities[n]} has no entry')
            
        else:
            data_votes = pd.DataFrame(data=[rows_votes]).T.rename({0:'candidate'}, axis=1)
            data_incumbent = data_votes[data_votes['candidate'].astype(str).str.contains(',')].reset_index(drop=True)
            data_incumbent['incumbent'] = data_incumbent['candidate'].astype(str).str.replace('\n</td>', '').str.replace('<td>', '')
            data_incumbent['party'] = data_votes[data_votes['candidate'].astype(str).str.contains(r'[A-Z][A-Z]')].reset_index(drop=True)
            data_incumbent['party'] = data_incumbent['party'].astype(str).str.replace('\n</td>', '').str.replace('<td>', '')
            data_incumbent['incumbent'] = data_incumbent['incumbent'].astype(str).str.replace('ß', 'ss').str.replace('Ü', 'U').str.replace('ü', 'ue').str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('.', '').str.replace('é', 'e')
#             data_votes['candidates'] = data_votes['candidate'].str.extract(r'([A-Z][a-z]+[\s][A-Z][a-z]+[-]?[A-Z]*[a-z]*[,]*[\s]*[A-Z]*[a-z]*)')
            
#             data_votes_2['votes'] = data_votes[data_votes["votes"].str.contains('<', na=False)].reset_index(drop=True)['votes'].str.replace('<', '')
#             data_votes_2['candidates'] = data_votes['candidates'].dropna().reset_index(drop=True)
            data_incumbent['city'] = list_cities[n]
            data_incumbent['year'] = 2016
            data_2016_incumbent = data_2016_incumbent.append(data_incumbent).dropna().reset_index(drop=True)

In [ ]:
data_2016_incumbent.rename({'candidate':'candidate_raw', 'incumbent':'candidate'},axis=1, inplace=True)

In [ ]:
data_2016_incumbent.to_csv(r'C:\Users\mariu\Downloads\hessen_inc_2016.csv')

In [ ]:
data_inc = data_2006_incumbent.append(data_2011_incumbent)
data_inc = data_inc.append(data_2016_incumbent)
data_inc.to_csv(r'C:\Users\mariu\Downloads\data_hessen_inc.csv')

In [ ]:
data_scrape['Gemeindename'] = data_scrape['Gemeindename'].str.replace('ÃŸ', 'ß').str.replace('Ã¼', 'ü').str.replace('Ã¶', 'ö').str.replace('Ã¤', 'ä')


In [ ]:
data_hessen_2 = data_hessen_2.rename({'city':'Gemeindename'},axis=1)
data_hessen_2['Gemeindename'] = data_hessen_2['Gemeindename'].str.replace('ÃŸ', 'ß').str.replace('Ã¼', 'ü').str.replace('Ã¶', 'ö').str.replace('Ã¤', 'ä')
data_hessen_3 = pd.merge(data_scrape[['Gemeindename', 'plz']], data_hessen_2, on=['Gemeindename'], how='outer', validate='1:m')
data_hessen_3

In [ ]:
data_hessen_3 = data_hessen_3.dropna()
data_hessen_3

In [ ]:
data_hessen_3['party'] = data_hessen_3.candidates.str.split(',', expand=True)[1]
data_hessen_3['candidate'] = data_hessen_3.candidates.str.split(',', expand=True)[0]
data_hessen_3

In [ ]:
party_drop_can = ['Christlich', 'Sozialdemokratische', 'Summe Kandidaten-Stimmen', 'Freie Demokratische', 'Piratenpartei',
                 'Buergerliste', 'Waehlervereinigung']

for name in party_drop_can:
    data_hessen_3 = data_hessen_3[~data_hessen_3['candidate'].str.contains(name)]

In [ ]:
data_hessen_3['doctor'] = data_hessen_3['candidate'].str.extract(r'(Dr )')
data_hessen_3['doctor'] = data_hessen_3.doctor.str.replace(r'(Dr )', '1')
data_hessen_3['candidate'] = data_hessen_3['candidate'].str.replace(r'(Dr )', '')
data_hessen_3['doctor'] = data_hessen_3['doctor'].astype(float)
data_hessen_3

In [ ]:
data_hessen_3['doctor'] = data_hessen_3['doctor'].fillna(0)
for i in range(1, len(data_hessen_3)-1):
    
    if (data_hessen_3.iloc[i-1, 5] == data_hessen_3.iloc[i+1, 5]) & (data_hessen_3.iloc[i,5] == None):
        data_hessen_3.iloc[i, 5] = data_hessen_3.iloc[i-1, 5]
        
    elif (data_hessen_3.iloc[i-1, 5] != data_hessen_3.iloc[i+1, 5]) &(data_hessen_3.iloc[i,5] == None):
        print(i)


In [ ]:
data_hessen_4 = pd.DataFrame()
for j in range(len(data_hessen_3['Gemeindename'].unique())):
    
    data_city = data_hessen_3[data_hessen_3['Gemeindename'] == data_hessen_3['Gemeindename'].unique()[j]]
    
    for t in range(len(data_city['year'].unique())):
        
        data_year = data_city[data_city['year'] == data_city['year'].unique()[t]]
        
        for k in range(len(data_year['party'].unique())):
            
            data_party = data_year[data_year['party'] == data_year['party'].unique()[k]].reset_index(drop=True)
            data_party['place_list'] = data_party.reset_index()['index']
            data_hessen_4 = data_hessen_4.append(data_party)
        

In [ ]:
data_hessen_4['party'] = data_hessen_4.party.str.replace(' Christlich', 'CDU').str.replace(' Sozialdemokratisch', 'SPD').str.replace(' BUNDNIS', 'Grüne').str.replace(' DIE', 'Linke').str.replace(' Freie', 'FDP').str.replace('Alternative', 'AfD')
data_hessen_4['party'] = data_hessen_4.party.str.replace(' ', '')
data_hessen_4['party'] = data_hessen_4.party.str.replace('GRUNE', 'Grüne').str.replace('SPDe', 'SPD').str.replace('Die', 'Linke').str.replace('Nationaldemokratische', 'NPD').str.replace('CDUe', 'CDU').str.replace('Gruen', 'Grüne')
data_hessen_4['party'] = data_hessen_4.party.str.replace('PARTEI', 'Partei').str.replace('Grünee', 'Grüne')
data_hessen_4['place_list'] = data_hessen_4['place_list'] + 1

In [ ]:
data_hessen_4['first_name']=data_hessen_4['candidate'].str.split(' ', expand=True)[0]
data_hessen_4['last_letter'] = data_hessen_4.first_name.str[-1]
data_hessen_4['pre_last_letter'] = data_hessen_4.first_name.str[-2]
data_hessen_4['third_last_letter'] = data_hessen_4.first_name.str[-3]
data_hessen_4['fourth_last_letter'] = data_hessen_4.first_name.str[-4]
data_hessen_4['fifth_last_letter'] = data_hessen_4.first_name.str[-5]
data_hessen_4['sixth_last_letter'] = data_hessen_4.first_name.str[-6]
data_hessen_4['seventh_last_letter'] = data_hessen_4.first_name.str[-7]
data_hessen_4['eigth_last_letter'] = data_hessen_4.first_name.str[-8]

In [ ]:
url = 'https://raw.githubusercontent.com/mariusgruenewald/local_election_predictions/master/data/Bavaria_2020/bayern_complete_1%20(5).csv'
data_train = pd.read_csv(url, sep=';', encoding='utf-8')
data_train = data_train[['Male', 'first_name']]
data_train['last_letter'] = data_train.first_name.str[-1]
data_train['pre_last_letter'] = data_train.first_name.str[-2]
data_train['third_last_letter'] = data_train.first_name.str[-3]
data_train['fourth_last_letter'] = data_train.first_name.str[-4]
data_train['fifth_last_letter'] = data_train.first_name.str[-5]
data_train['sixth_last_letter'] = data_train.first_name.str[-6]
data_train['seventh_last_letter'] = data_train.first_name.str[-7]
data_train['eigth_last_letter'] = data_train.first_name.str[-8]

In [ ]:
# get_dummies can handle missing values
list_columns = data_train.columns[2:]
data_train_2 = pd.get_dummies(data_train.iloc[:, 2:], prefix=list_columns)
data_test = pd.get_dummies(data_hessen_4[list_columns], prefix=list_columns)

In [ ]:
data_test = pd.concat([data_hessen_4[['first_name', 'year', 'Gemeindename', 'party']], data_test], axis=1)

In [ ]:
data_train = pd.concat([data_train[['Male', 'first_name']], data_train_2],axis=1)
data_train

In [ ]:
data_train = data_train.dropna()
data_train

In [ ]:
columns_test = data_test.columns[4:]
columns_train = data_train.columns[2:]

joint_var = columns_test.intersection(columns_train)
joint_var

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

param_grid_1 = {
    'max_depth': range(10,100,10),
    'min_samples_leaf': [4, 8, 12, 16, 20],
    'min_samples_split': [10,20,30,40,50],
    'n_estimators': range(10,100,10),
}

random_forest = GridSearchCV(estimator = RandomForestClassifier()
                             , param_grid = param_grid_1, cv = 5, verbose = 1, n_jobs = -1)
t2 = time.time()
random_forest.fit(data_train[joint_var], data_train['Male'])
t_random_forest = time.time() - t2


random_forest.score(data_train.iloc[:,2:],data_train['Male']), t_random_forest

In [ ]:
random_forest.best_params_

In [ ]:
rf = RandomForestClassifier(max_depth= 80, min_samples_leaf= 4, min_samples_split= 10, n_estimators = 90)
rf.fit(data_train[joint_var],data_train['Male'])

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score


def display_score(reg, mse_reg, r2):
    
    """ Insert explanation """
    
    reg_rmse = np.sqrt(-mse_reg)
    
    print('MSE                                  ')
    print('Scores:', reg_rmse, reg)
    print('Mean:', reg_rmse.mean(), reg)
    print('standard Deviation:', reg_rmse.std(), reg)
    print('R2                                   ')
    print('Scores:', r2, reg)
    print('Mean:', r2.mean(), reg)
    print('standard Deviation:', r2.std(), reg)

In [ ]:
mse_rf_train = cross_val_score(rf, data_train[joint_var],data_train['Male'], scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_rf_train = cross_val_score(rf, data_train[joint_var],data_train['Male'], scoring = 'r2', cv = 10, n_jobs = -1)
display_score("RF", mse_rf_train, r2_rf_train)

In [ ]:
rf_reg_pred = pd.DataFrame(rf.predict(data_test[joint_var]), columns=['Male'])
rf_reg_pred_probs = pd.DataFrame(rf.predict_proba(data_test[joint_var]), columns=['Probability_fem', 'Probability_male'])
rf_pred = pd.concat([rf_reg_pred_probs, rf_reg_pred], axis=1).reset_index(drop=True)
rf_pred_3 = pd.concat([rf_pred, data_hessen_4.reset_index(drop=True)], axis=1)

In [ ]:
data_hessen_5 = rf_pred_3.drop(['last_letter', 'pre_last_letter', 'third_last_letter', 'fourth_last_letter', 'fifth_last_letter', 'sixth_last_letter', 'seventh_last_letter', 'eigth_last_letter'], axis=1)
data_hessen_5 # now: manually double check Gender prediction

In [ ]:
float_col = data_hessen_5.loc[:, data_hessen_5.dtypes == float].columns
for j in float_col:
    data_hessen_5[j] = data_hessen_5[j].astype(int)
data_hessen_5

In [ ]:
data_hessen_5.to_csv(r'C:\Users\mariu\documents\data_hessen_1.csv')

In [ ]:
data_hessen_6 = pd.read_csv(r'https://raw.githubusercontent.com/mariusgruenewald/local_election_predictions/master/data/hessen_2021/data_hessen_1.csv', delimiter=';')
# done: manually double check Gender prediction

In [ ]:
data_hessen_6

In [ ]:
data_hessen_6['muslim'] = 0
data_hessen_6.loc[(data_hessen_6['first_name'] == 'Ali') | (data_hessen_6['first_name'] == 'Hüseyin') | 
                (data_hessen_6['first_name'] == 'Cumali') | (data_hessen_6['first_name'] == 'Nükhet') | 
                (data_hessen_6['first_name'] == 'Gökhan')
                | (data_hessen_6['first_name'] == 'Serafettin') | (data_hessen_6['first_name'] == 'Zeynep')
                | (data_hessen_6['first_name'] == 'Aysel') | (data_hessen_6['first_name'] == 'Gönül') 
                | (data_hessen_6['first_name'] == 'Ayten') | (data_hessen_6['first_name'] == 'Gülseren') 
                | (data_hessen_6['first_name'] == 'Ilknur') | (data_hessen_6['first_name'] == 'Kenan')
                | (data_hessen_6['first_name'] == 'Cetin') | (data_hessen_6['first_name'] == 'Kerem')
                | (data_hessen_6['first_name'] == 'Oguz') | (data_hessen_6['first_name'] == 'Ilhan')
                | (data_hessen_6['first_name'] == 'Asya') | (data_hessen_6['first_name'] == 'Tuncay')
                | (data_hessen_6['first_name'] == 'Hakan') | (data_hessen_6['first_name'] == 'Bülent')
                | (data_hessen_6['first_name'] == 'Cem') | (data_hessen_6['first_name'] == 'Yasar')
                | (data_hessen_6['first_name'] == 'Ilknur') | (data_hessen_6['first_name'] == 'Orhan')
                | (data_hessen_6['first_name'] == 'Resat') | (data_hessen_6['first_name'] == 'Magdi')
                | (data_hessen_6['first_name'] == 'Teslime') | (data_hessen_6['first_name'] == 'Uygur')
                | (data_hessen_6['first_name'] == 'Sedef') | (data_hessen_6['first_name'] == 'Ahmet')
                | (data_hessen_6['first_name'] == 'Mercan') | (data_hessen_6['first_name'] == 'Aziz')
                | (data_hessen_6['first_name'] == 'Ismail') | (data_hessen_6['first_name'] == 'Ümit')
                | (data_hessen_6['first_name'] == 'Yasemin') | (data_hessen_6['first_name'] == 'Aynur')
                | (data_hessen_6['first_name'] == 'Abdurrahman') | (data_hessen_6['first_name'] == 'Süleyman')
                | (data_hessen_6['first_name'] == 'Mustafa') | (data_hessen_6['first_name'] == 'Senem')
                | (data_hessen_6['first_name'] == 'Özlem') | (data_hessen_6['first_name'] == 'Taylan')
                | (data_hessen_6['first_name'] == 'Zeki') | (data_hessen_6['first_name'] == 'Nazile')
                | (data_hessen_6['first_name'] == 'Ümran') | (data_hessen_6['first_name'] == 'Ferhat')
                | (data_hessen_6['first_name'] == 'Neslihan') | (data_hessen_6['first_name'] == 'Maksude')
                | (data_hessen_6['first_name'] == 'Fadime') | (data_hessen_6['first_name'] == 'Güray')
                | (data_hessen_6['first_name'] == 'Metin') | (data_hessen_6['first_name'] == 'Ramazan')
                | (data_hessen_6['first_name'] == 'Bediye') | (data_hessen_6['first_name'] == 'Hasim')
                | (data_hessen_6['first_name'] == 'Bekir') | (data_hessen_6['first_name'] == 'Osman')
                | (data_hessen_6['first_name'] == 'Gülten') | (data_hessen_6['first_name'] == 'Refiye')
                | (data_hessen_6['first_name'] == 'Hatun') | (data_hessen_6['first_name'] == 'Arif')
                | (data_hessen_6['first_name'] == 'Hassan') | (data_hessen_6['first_name'] == 'Celal')
                | (data_hessen_6['first_name'] == 'Ibrahim') | (data_hessen_6['first_name'] == 'Sevim')
                | (data_hessen_6['first_name'] == 'Sait') | (data_hessen_6['first_name'] == 'Yetvart')
                | (data_hessen_6['first_name'] == 'Ugur') | (data_hessen_6['first_name'] == 'Mehmet')
                | (data_hessen_6['first_name'] == 'Cemal') | (data_hessen_6['first_name'] == 'Tülay')
                | (data_hessen_6['first_name'] == 'Muhammet') | (data_hessen_6['first_name'] == 'Ayaz')
                | (data_hessen_6['first_name'] == 'Mahbubur') | (data_hessen_6['first_name'] == 'Durdu')
                | (data_hessen_6['first_name'] == 'Tugay') | (data_hessen_6['first_name'] == 'Cengiz')
                | (data_hessen_6['first_name'] == 'Ekrem') | (data_hessen_6['first_name'] == 'Zehra')
                | (data_hessen_6['first_name'] == 'Hüsamettin') | (data_hessen_6['first_name'] == 'Müslüm')
                | (data_hessen_6['first_name'] == 'Kevser') | (data_hessen_6['first_name'] == 'Baki')
                | (data_hessen_6['first_name'] == 'Ikbal') | (data_hessen_6['first_name'] == 'Nejdet')
                | (data_hessen_6['first_name'] == 'Güllü') | (data_hessen_6['first_name'] == 'Fikret')
                | (data_hessen_6['first_name'] == 'Handan') | (data_hessen_6['first_name'] == 'Tezcan')
                | (data_hessen_6['first_name'] == 'Gülhan') | (data_hessen_6['first_name'] == 'Özel')
                | (data_hessen_6['first_name'] == 'Hemdegül') | (data_hessen_6['first_name'] == 'Nail')
                | (data_hessen_6['first_name'] == 'Babür') | (data_hessen_6['first_name'] == 'Deniz')
                | (data_hessen_6['first_name'] == 'Songül') | (data_hessen_6['first_name'] == 'Nebil')
                | (data_hessen_6['first_name'] == 'Alper') | (data_hessen_6['first_name'] == 'Gülay')
                | (data_hessen_6['first_name'] == 'Dilan') | (data_hessen_6['first_name'] == 'Nebil')
                | (data_hessen_6['first_name'] == 'Birgül') | (data_hessen_6['first_name'] == 'Melihat')
                | (data_hessen_6['first_name'] == 'Devrim') | (data_hessen_6['first_name'] == 'Arzu')
                | (data_hessen_6['first_name'] == 'Erkan') | (data_hessen_6['first_name'] == 'Nesrin')
                | (data_hessen_6['first_name'] == 'Ercan') | (data_hessen_6['first_name'] == 'Abdulmesih')
                | (data_hessen_6['first_name'] == 'Ayhan') | (data_hessen_6['first_name'] == 'Aydin')
                | (data_hessen_6['first_name'] == 'Kamran') | (data_hessen_6['first_name'] == 'Munib')
                | (data_hessen_6['first_name'] == 'Hamed') | (data_hessen_6['first_name'] == 'Çigdem')
                | (data_hessen_6['first_name'] == 'Hülya') | (data_hessen_6['first_name'] == 'Zeynel')
                | (data_hessen_6['first_name'] == 'Zafer') | (data_hessen_6['first_name'] == 'Dalal')
                | (data_hessen_6['first_name'] == 'Cafer') | (data_hessen_6['first_name'] == 'Yasin')
                | (data_hessen_6['first_name'] == 'Batoul') | (data_hessen_6['first_name'] == 'Murat')
                | (data_hessen_6['first_name'] == 'Halil') | (data_hessen_6['first_name'] == 'Gülcin')
                | (data_hessen_6['first_name'] == 'Zafar') | (data_hessen_6['first_name'] == 'Sanjar')
                | (data_hessen_6['first_name'] == 'Tugba') | (data_hessen_6['first_name'] == 'Raziye')
                | (data_hessen_6['first_name'] == 'Aliye') | (data_hessen_6['first_name'] == 'Niyazi')
                | (data_hessen_6['first_name'] == 'Kemal') | (data_hessen_6['first_name'] == 'Özgür')
                | (data_hessen_6['first_name'] == 'Veysel') | (data_hessen_6['first_name'] == 'Khaled')
                | (data_hessen_6['first_name'] == 'Idris') | (data_hessen_6['first_name'] == 'Yilmaz')
                | (data_hessen_6['first_name'] == 'Önder') | (data_hessen_6['first_name'] == 'Tülin')
                | (data_hessen_6['first_name'] == 'Göktürk') | (data_hessen_6['first_name'] == 'Sadik')
                | (data_hessen_6['first_name'] == 'Selvet') | (data_hessen_6['first_name'] == 'Abdellatif')
                | (data_hessen_6['first_name'] == 'Hava') | (data_hessen_6['first_name'] == 'Sengül')
                | (data_hessen_6['first_name'] == 'Erhan') | (data_hessen_6['first_name'] == 'Ergün')
                | (data_hessen_6['first_name'] == 'Serdar') | (data_hessen_6['first_name'] == 'Yunus')
                | (data_hessen_6['first_name'] == 'Nurullah') | (data_hessen_6['first_name'] == 'Nida')
                | (data_hessen_6['first_name'] == 'Nursan') | (data_hessen_6['first_name'] == 'Berkkan')
                | (data_hessen_6['first_name'] == 'Ertürk') | (data_hessen_6['first_name'] == 'Sakine')
                | (data_hessen_6['first_name'] == 'Ufuk') | (data_hessen_6['first_name'] == 'Kasim')
                | (data_hessen_6['first_name'] == 'Lütfi') | (data_hessen_6['first_name'] == 'Emrah')
                | (data_hessen_6['first_name'] == 'Gürsoy') | (data_hessen_6['first_name'] == 'Yusuf')
                | (data_hessen_6['first_name'] == 'Maryam') | (data_hessen_6['first_name'] == 'Rabiye')
                | (data_hessen_6['first_name'] == 'Gülbeyaz') | (data_hessen_6['first_name'] == 'Nezaket')
                | (data_hessen_6['first_name'] == 'Riza') | (data_hessen_6['first_name'] == 'Görkan')
                | (data_hessen_6['first_name'] == 'Mohamad') | (data_hessen_6['first_name'] == 'Mohammad')
                | (data_hessen_6['first_name'] == 'Yavuz') | (data_hessen_6['first_name'] == 'Hachem')
                | (data_hessen_6['first_name'] == 'Kaan') | (data_hessen_6['first_name'] == 'Erdal')
                , 'muslim' ] = 1

In [ ]:
data_hessen_6['last_name'] = data_hessen_6.candidate.str.split(' ', expand=True)[1]
data_hessen_6

In [ ]:
data_hessen_6['last_letter'] = data_hessen_6.last_name.str[-1]
data_hessen_6['pre_last_letter'] = data_hessen_6.last_name.str[-2]
data_hessen_6['third_last_letter'] = data_hessen_6.last_name.str[-3]
data_hessen_6['fourth_last_letter'] = data_hessen_6.last_name.str[-4]
data_hessen_6['fifth_last_letter'] = data_hessen_6.last_name.str[-5]
data_hessen_6['sixth_last_letter'] = data_hessen_6.last_name.str[-6]
data_hessen_6['seventh_last_letter'] = data_hessen_6.last_name.str[-7]
data_hessen_6['eigth_last_letter'] = data_hessen_6.last_name.str[-8]

data_hessen_6['last_letter_first_name'] = data_hessen_6.first_name.str[-1]
data_hessen_6['pre_last_letter_first_name'] = data_hessen_6.first_name.str[-2]
data_hessen_6['third_last_letter_first_name'] = data_hessen_6.first_name.str[-3]

In [ ]:
data_hessen_6['non_muslim_migrant'] = 0

# Greek name structure
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'u') & (data_hessen_6['pre_last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'i')
                & (data_hessen_6['third_last_letter'] == 'd') & (data_hessen_6['fourth_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'i')
                & (data_hessen_6['third_last_letter'] == 't') & (data_hessen_6['fourth_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'i')
                & (data_hessen_6['third_last_letter'] == 'd') & (data_hessen_6['fourth_last_letter'] == 'a'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                & (data_hessen_6['third_last_letter'] == 's') & (data_hessen_6['fourth_last_letter'] == 't')
                , 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                & (data_hessen_6['third_last_letter'] == 'l') & (data_hessen_6['fourth_last_letter'] == 'u')
                , 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                & (data_hessen_6['third_last_letter'] == 'r'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                & (data_hessen_6['third_last_letter'] == 'k'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                & (data_hessen_6['third_last_letter'] == 's'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                & (data_hessen_6['third_last_letter'] == 'n'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'a')
                & (data_hessen_6['third_last_letter'] == 'r'), 'non_muslim_migrant' ] = 1

data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i') & (data_hessen_6['pre_last_letter'] == 'k')
                & (data_hessen_6['third_last_letter'] == 's'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'y') & (data_hessen_6['pre_last_letter'] == 'k')
                & (data_hessen_6['third_last_letter'] == 's'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'k') & (data_hessen_6['pre_last_letter'] == 'y')
                & (data_hessen_6['third_last_letter'] == 'z'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'k') & (data_hessen_6['pre_last_letter'] == 'a')
                & (data_hessen_6['third_last_letter'] == 'z')
                , 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'z') & (data_hessen_6['pre_last_letter'] == 'c')
                & (data_hessen_6['third_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'k') & (data_hessen_6['pre_last_letter'] == 'e')
                , 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'c') & (data_hessen_6['pre_last_letter'] == 'i')
                & (data_hessen_6['third_last_letter'] == 'v'), 'non_muslim_migrant' ] = 1

data_hessen_6.loc[(data_hessen_6['last_letter_first_name'] == 'v') & (data_hessen_6['pre_last_letter_first_name'] == 'a')
                & (data_hessen_6['third_last_letter_first_name'] == 'l'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter_first_name'] == 'w') & (data_hessen_6['pre_last_letter_first_name'] == 'e')
                & (data_hessen_6['third_last_letter_first_name'] == 'j'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'y') & (data_hessen_6['pre_last_letter'] == 'n')
                , 'non_muslim_migrant' ] = 1

data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i') & (data_hessen_6['pre_last_letter'] == 'n')
                & (data_hessen_6['third_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'a') & (data_hessen_6['pre_last_letter'] == 'i')
                , 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i') & (data_hessen_6['pre_last_letter'] == 'n')
                & (data_hessen_6['third_last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'e') & (data_hessen_6['pre_last_letter'] == 'n')
                & (data_hessen_6['third_last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
data_hessen_6.loc[(data_hessen_6['last_letter'] == 'e') & (data_hessen_6['pre_last_letter'] == 's')
                & (data_hessen_6['third_last_letter'] == 'e'), 'non_muslim_migrant' ] = 1

data_hessen_6.loc[(data_hessen_6['last_name'] == 'Krone'), 'non_muslim_migrant'] = 0

data_hessen_6 = data_hessen_6.drop(['last_letter_first_name', 'pre_last_letter_first_name', 'third_last_letter_first_name', 
                                'last_letter', 'pre_last_letter', 'third_last_letter', 'fourth_last_letter', 'fifth_last_letter', 'sixth_last_letter',
                 'seventh_last_letter', 'eigth_last_letter'], axis = 1)

data_hessen_6

In [ ]:
data_hessen_6 = data_hessen_6[['Probability_fem','Probability_male','Male','Gemeindename','plz','votes','candidates','year','party','candidate','doctor','place_list','first_name','muslim','last_name','non_muslim_migrant']]
data_hessen_6

In [ ]:
data_inc = pd.read_csv(r'C:\Users\mariu\Documents\inc.csv', sep=';', encoding='utf-8')
data_inc = data_inc.iloc[:, 2:]
#data_inc = data_inc.dropna(subset=['incumbent']).reset_index(drop=True)
data_inc.rename({'Gemeindename':'city'},axis=1, inplace=True)
data_inc.loc[data_inc['party'].str.contains('FW', na=False), 'party'] = 'FW'

In [ ]:
data_2016_incumbent['incumbent'] = 1
data_2016_incumbent['candidate'] = data_2016_incumbent['candidate'].str.replace('Dr ', '').str.replace('Prof ', '').str.replace('med ', '')
data_2016_incumbent['city'] = data_2016_incumbent['city'].str.replace('Ã¼', 'ü').str.replace('ÃŸ', 'ß').str.replace('Ã¶', 'ö').str.replace('Ã¤', 'ä')
data_2016_incumbent['first_name'] = data_2016_incumbent['candidate'].str.split(', ',expand =True)[1]
data_2016_incumbent['last_name'] = data_2016_incumbent['candidate'].str.split(', ',expand =True)[0]
data_2016_incumbent['party'] = data_2016_incumbent.party.str.replace('GRÜNE', 'Grüne').str.replace('DIE LINKE', 'Linke').str.replace('PARTEI', 'Partei').str.replace('Grünee', 'Grüne')
data_2016_incumbent.loc[data_2016_incumbent['party'].str.contains('FW'), 'party'] = 'FW'
data_2016_incumbent

In [ ]:
data_inc_test = pd.merge(data_inc, data_2016_incumbent, on=['year', 'party', 'city', 'last_name', 'first_name'], how='left', validate='m:m')
data_inc_test.drop(['candidate_raw', 'candidate_y'], axis=1, inplace=True)

In [ ]:
data_inc_test.loc[(data_inc_test['incumbent_y'].isna()) & (data_inc_test['incumbent_x'] == 0), 'incumbent_y'] = 0
data_inc_test[~data_inc_test['incumbent_y'].isna() & data_inc_test['incumbent_x'].isna()]

for city in data_inc_test['city'].unique():
    data_city = data_inc_test[data_inc_test['city'] == city]
    
    if any(~data_city['incumbent_y'].isna()):
        data_inc_test.loc[(data_inc_test['city'] == city) & (data_inc_test['incumbent_y'].isna()), 'incumbent_y'] = 0
        
data_inc_test

In [ ]:
data_hessen_hist = data_inc_test.dropna(subset=['incumbent_y']).drop(['incumbent_x'], axis=1)
data_hessen_hist.rename({'candidate_x': 'candidate', 'incumbent_y':'incumbent'}, axis=1, inplace=True)

In [ ]:
data_hessen_hist

In [ ]:
data_hessen_marius = pd.read_csv(r'C:\Users\mariu\Documents\limburg.csv', sep=';', encoding='utf-8')
link_thomas = 'https://raw.githubusercontent.com/mariusgruenewald/local_election_predictions/master/data/hessen_2021/Candidates_Thomas.csv'
data_hessen_thomas = pd.read_csv(link_thomas, sep=',', encoding='utf-8')
data_hessen_edith = pd.read_csv(r'C:\Users\mariu\Documents\hessen_edith.csv', sep=';', encoding='utf-8')
data_hessen_edith

In [ ]:
data_hessen_thomas.rename({'Geburtsjahr':'birth_year', 'Partei':'party', 'Geburtsort':'birth_place', 'Nachname':'last_name', 'Vorname':'first_name', 'Beruf':'job', 'Adresse':'address', 'Wohnort':'city'},
                          axis=1, inplace=True)
data_hessen_edith.rename({'place_residence':'city', 'number':'place_list', 'occupation':'job'}, axis=1, inplace=True)
data_hessen_marius.rename({'title':'sex', 'adress':'address'},  axis=1, inplace=True)

In [ ]:
data_hessen_marius['doctor'] = 0
data_hessen_marius.loc[data_hessen_marius['sex'].str.contains('Dr.', na=False), 'doctor'] = 1
data_hessen_marius['sex'] = data_hessen_marius.sex.str.replace('Dr.', '').str.replace('Prof. ', '')
data_hessen_edith['doctor'] = 0
data_hessen_edith.loc[(data_hessen_edith['last_name'].str.contains('Dr.')) | (data_hessen_edith['first_name'].str.contains('Dr.')), 'doctor'] = 1
data_hessen_thomas['doctor'] = 0
data_hessen_thomas.loc[(data_hessen_thomas['last_name'].str.contains('Dr.')) | (data_hessen_thomas['first_name'].str.contains('Dr.')) | (data_hessen_thomas['last_name'].str.contains('Dr ')) | (data_hessen_thomas['first_name'].str.contains('Dr ')), 'doctor'] = 1
data_hessen_thomas.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
data_hessen_marius['birth_year'] = data_hessen_marius['birth_year'].str.replace('geb. ', '').str.replace(' in', '').str.replace('\xa0', '').str.replace('954 i', '1954').str.replace(' i', '').str.replace('geb.', '').str.replace('geb', '')
data_hessen_marius['birth_year'] = data_hessen_marius['birth_year'].str.replace(' Fr', '').str.replace('972', '1972').str.replace(' ', '').str.replace('111972', '1972').astype(float)
data_hessen_marius['birth_place'] = data_hessen_marius['birth_place'].str.replace(' in', '')
data_hessen_marius['Female'] = 0
data_hessen_marius.loc[data_hessen_marius['sex'].str.contains('Frau', na=False), 'Female'] = 1
data_hessen_edith['Female'] = 0
data_hessen_edith.loc[data_hessen_edith['sex'].str.contains('Frau', na=False), 'Female'] = 1

In [ ]:
data_hessen_2021 = data_hessen_marius.append(data_hessen_edith)

In [ ]:
data_hessen_thomas['last_name'] = data_hessen_thomas.last_name.str.replace('Dr. ', '').str.replace('Prof. ', '').str.replace('Dr ', '')
data_hessen_thomas['first_name'] = data_hessen_thomas.first_name.str.replace('Dr. ', '').str.replace('Prof. ', '').str.replace('Dr ', '')

data_hessen_thomas_2= pd.DataFrame()
for city in data_hessen_thomas['city'].unique():
    data_city = data_hessen_thomas[data_hessen_thomas['city'] == city]
    
    for party in data_city['party'].unique():
        data_party = data_city[data_city['party'] == party]
        data_party = data_party.reset_index(drop=True).reset_index()
        
        data_hessen_thomas_2 = data_hessen_thomas_2.append(data_party)
        
data_hessen_thomas_2 = data_hessen_thomas_2.reset_index(drop=True)
data_hessen_thomas_2.rename({'index':'place_list'},axis=1, inplace=True)

In [ ]:
data_hessen_2021 = data_hessen_2021.append(data_hessen_thomas_2)
data_hessen_2021.reset_index(drop=True,inplace=True)

In [ ]:
data_hessen_2021['first_name'] = data_hessen_2021.first_name.str.replace(r'^[\s]', '')
list_male_names = data_hessen_2021[data_hessen_2021['Female'] == 0]['first_name'].unique()
list_female_names = data_hessen_2021[data_hessen_2021['Female'] == 1]['first_name'].unique()

data_hessen_2021.loc[(data_hessen_2021['first_name'].isin(list_male_names)) & (data_hessen_2021['Female'].isna()), 'Female'] = 0
data_hessen_2021.loc[(data_hessen_2021['first_name'].isin(list_male_names)) & (data_hessen_2021['Female'].isna()), 'Female'] = 1

In [ ]:
data_hessen_2021['last_name'] = data_hessen_2021.last_name.str.replace(r'^[0-9]*[\s]*', '')

In [ ]:
data_hessen_2021.loc[(data_hessen_2021['Female'].isna()) & (data_hessen_2021['first_name'].str.contains(r'a$')), 'Female'] =1
data_hessen_2021.loc[(data_hessen_2021['Female'].isna()) & (data_hessen_2021['first_name'].str.contains(r'e$')), 'Female'] =1
data_hessen_2021.loc[(data_hessen_2021['Female'].isna()) & (data_hessen_2021['first_name'].str.contains(r'i$')), 'Female'] =1
data_hessen_2021.loc[(data_hessen_2021['Female'].isna()) & (data_hessen_2021['first_name'].str.contains(r'us$')), 'Female'] = 0
data_hessen_2021.loc[(data_hessen_2021['Female'].isna()) & (data_hessen_2021['first_name'].str.contains(r'und$')), 'Female'] = 0
data_hessen_2021.loc[(data_hessen_2021['Female'].isna()) & (data_hessen_2021['first_name'].str.contains(r'ard$')), 'Female'] = 1
data_hessen_2021.loc[(data_hessen_2021['Female'].isna()) & (data_hessen_2021['first_name'].str.contains(r'Dagmar')), 'Female'] = 1
# do ~200 by hand (accuracy)
data_hessen_2021[data_hessen_2021['Female'].isna()]
data_hessen_2021['year'] = 2021

In [ ]:
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('CDU'), 'party'] = 'CDU'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('LINKE'), 'party'] = 'Die Linke'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('Linke'), 'party'] = 'Die Linke'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('SPD'), 'party'] = 'SPD'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('GRÜNE'), 'party'] = 'Grüne'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('Gruene'), 'party'] = 'Grüne'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('Grünen'), 'party'] = 'Grüne'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('AFD'), 'party'] = 'AfD'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('AfD'), 'party'] = 'AfD'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('FW'), 'party'] = 'FW'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('Fraktion Riedstädter Bürger FRB'), 'party'] = 'FRB'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('Bürger für Riedstadt BfR'), 'party'] = 'BfR'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('Freie Wähler'), 'party'] = 'FW'
data_hessen_2021.loc[data_hessen_2021['party'].str.contains('Freie Waehler'), 'party'] = 'FW'
data_hessen_2021.loc[data_hessen_2021['city'].str.contains('Frankenberg', na=False), 'city'] = 'Frankenberg (Eder)'


In [ ]:
# missing incumbent data: Kassel, Hattersheim, Muehlheim, Griesheim
data_hessen_2021['city'] = data_hessen_2021['city'].str.replace('Neu-Isendorf', 'Neu-Isenburg').str.replace(r'^[\s]', '').str.replace('Bad Nauheim', 'Bad Nauheim')

In [ ]:
data_hessen_2021['city'] = data_hessen_2021.city.str.replace(r'[0-9]+', '').str.replace(r'^[\s]', '')
data_hessen_2021.loc[data_hessen_2021['city'].str.contains('Idstein-', na=False), 'city'] = 'Idstein'
data_hessen_2021['city'] = data_hessen_2021['city'].str.replace('Bad\xa0Nauheim', 'Bad Nauheim').str.replace('Gries-heim ', 'Griesheim').str.replace('Griesheim ', 'Griesheim').str.replace('BadHersfeld', 'Bad Hersfeld')
data_hessen_2021['city'] = data_hessen_2021['city'].str.replace('ß', 'ss').str.replace('ü', 'ue').str.replace('ö', 'oe').str.replace('\xa0', '').str.replace('Hofheim', 'Hofheim am Taunus')
data_hessen_2021['city'] = data_hessen_2021['city'].str.replace('Langen', 'Langen (Hessen)')
data_hessen_2021['city'].unique()

In [ ]:
data_inc_2021 = pd.read_csv(r'C:\Users\mariu\Downloads\hessen_inc_2021.csv',engine='python', encoding='utf-8')
data_inc_2021 = data_inc_2021[['incumbent', 'party', 'city', 'year']]
data_inc_2021['last_name'] = data_inc_2021.incumbent.str.split(', ', expand=True)[0]
data_inc_2021['first_name'] = data_inc_2021.incumbent.str.split(', ', expand=True)[1]

data_inc_2021['last_name'] = data_inc_2021['last_name'].str.replace('Dr ', '').str.replace('Prof ', '').str.replace('-Ing ', '')

data_inc_2021['city'] = data_inc_2021['city'].str.replace('Ã¼', 'ue').str.replace('ÃŸ', 'ss').str.replace('Ã¶', 'oe').str.replace('Ã¤', 'ae')
data_inc_2021.loc[(data_inc_2021['party'].str.contains('Freie Wähler')) | (data_inc_2021['party'].str.contains('FW')), 'party'] = 'FW'
data_inc_2021.loc[(data_inc_2021['party'].str.contains('GRÜNE', na=False)), 'party'] = 'Grüne'
data_inc_2021.loc[(data_inc_2021['party'].str.contains('LINKE', na=False)), 'party'] = 'Die Linke'

data_inc_2021.loc[(data_inc_2021['party'].str.contains('DL', na=False)), 'party'] = 'Dietzenbacher Liste'
data_inc_2021.loc[(data_inc_2021['party'].str.contains('BLW', na=False)), 'party'] = 'Bürgerliste Wiesbaden'
data_inc_2021.loc[(data_inc_2021['party'].str.contains('ULW', na=False)), 'party'] = 'Unabhängige Liste Wiesbaden'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Wiesbaden', na=False)) & (data_inc_2021['party'].isna()), 'party'] = 'AfD'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Hofheim', na=False)) & (data_inc_2021['party'] == 'WG Wallau'), 'party'] = 'Bürger für Hofheim'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Hofheim', na=False)) & (data_inc_2021['party'].isna()), 'party'] = 'WG Wallau'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Dietzenbach', na=False)) & (data_inc_2021['party'].isna()), 'party'] = 'AfD'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Moerfelden-Walldorf', na=False)) & (data_inc_2021['party'].isna()), 'party'] = 'FW'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Moerfelden-Walldorf', na=False)) & (data_inc_2021['party'] == 'DKP/LL'), 'party'] = 'Die Linke'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Marburg', na=False)) & (data_inc_2021['party'].isna()) & (data_inc_2021['last_name'].str.contains('Dr Weber')), 'party'] = 'Piraten'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Marburg', na=False)) & (data_inc_2021['party'] == 'PIRATEN'), 'party'] = 'Bürger für Marburg'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Marburg', na=False)) & (data_inc_2021['party'].isna()), 'party'] = 'Bürger für Marburg'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Hanau', na=False)) & (data_inc_2021.index >= 2568) & (data_inc_2021.index <= 2571), 'party'] = 'Bürger für Hanau'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Hanau', na=False)) & (data_inc_2021.index >= 2572) & (data_inc_2021.index <= 2576), 'party'] = 'REP'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Hanau', na=False)) & (data_inc_2021.index >= 2577) & (data_inc_2021.index <= 2578), 'party'] = 'ALL'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Hanau', na=False)) & (data_inc_2021.index == 2579), 'party'] = 'FGH'
data_inc_2021.loc[(data_inc_2021['city'] == 'Fulda') & (data_inc_2021['party'] == 'CWE'), 'party'] = 'Christliche Wähler-Einheit'
data_inc_2021.loc[(data_inc_2021['city'] == 'Fulda') & (data_inc_2021['party'] == 'REP'), 'party'] = 'Die Linke'
data_inc_2021.loc[(data_inc_2021['city'] == 'Fulda') & (data_inc_2021['party'].isna()), 'party'] = 'REP'
data_inc_2021.loc[(data_inc_2021['city'] == 'Darmstadt') & (data_inc_2021['party'] == 'UFFBASSE'), 'party'] = 'Unabhängige Fraktion Freier Bürger – Aufrecht, Spontan, Subkulturell, Eigensinnig '
data_inc_2021.loc[(data_inc_2021['city'] == 'Darmstadt') & (data_inc_2021['party'] == 'UWIGA'), 'party'] = 'Unabhängige Wählervereinigung, hervorgegangen aus der IG-Abwasser (UWIGA'
data_inc_2021.loc[(data_inc_2021['city'] == 'Darmstadt') & (data_inc_2021['party'].isna()), 'party'] = 'AfD'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Bensheim', na=False)) & (data_inc_2021.index >= 200) & (data_inc_2021.index <= 204), 'party'] = 'Bürger für Bensheim'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Bensheim', na=False)) & (data_inc_2021.index >= 206) & (data_inc_2021.index <= 211), 'party'] = 'AfD'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Bensheim', na=False)) & (data_inc_2021.index == 205), 'party'] = 'FW'
data_inc_2021.loc[(data_inc_2021['city'] == 'Neu-Isenburg') & (data_inc_2021['party'].isna()), 'party'] = 'AfD'
data_inc_2021.loc[(data_inc_2021['city'] == 'Koenigstein im Taunus') & (data_inc_2021['party'] == 'ALK'), 'party'] = 'Aktionsgemeinschaft Lebenswertes Königstein'
data_inc_2021.loc[(data_inc_2021['city'] == 'Kuenzell') & (data_inc_2021['party'] == 'CWE'), 'party'] = 'Christliche Wähler Einheit'
data_inc_2021.loc[(data_inc_2021['city'] == 'Eltville am Rhein') & (data_inc_2021['party'] == 'BL'), 'party'] = 'Bürgerlich Liberale Liste'
data_inc_2021.loc[(data_inc_2021['city'] == 'Bad Vilbel') & (data_inc_2021['party'].isna()), 'party'] = 'FW'
data_inc_2021.loc[(data_inc_2021['city'] == 'Roedermark') & (data_inc_2021['party'] == 'AL-Grüne'), 'party'] = 'Grüne'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Butzbach', na=False)) & (data_inc_2021.index >= 4632) & (data_inc_2021.index <= 4635), 'party'] = 'Grüne'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Butzbach', na=False)) & (data_inc_2021.index >= 4636) & (data_inc_2021.index <= 4640), 'party'] = 'FDP'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Butzbach', na=False)) & (data_inc_2021.index >= 4641) & (data_inc_2021.index <= 4649), 'party'] = 'UWG'
data_inc_2021.loc[(data_inc_2021['city'] == 'Weiterstadt') & (data_inc_2021['party'] == 'ALW'), 'party'] = 'ALW-Grüne'
data_inc_2021.loc[(data_inc_2021['city'] == 'Korbach') & (data_inc_2021['party'].isna()), 'party'] = 'Aktive Bürgerinnen und Bürger Korbach'
data_inc_2021.loc[(data_inc_2021['city'] == 'Bad Wildungen') & (data_inc_2021['party'].isna()), 'party'] = 'WfW'
data_inc_2021.loc[(data_inc_2021['city'] == 'Bad Soden am Taunus') & (data_inc_2021['party'] == 'BSB'), 'party'] = 'Bad Sodener Bürger'
data_inc_2021.loc[(data_inc_2021['city'] == 'Bad Soden am Taunus') & (data_inc_2021['party'].isna()), 'party'] = 'AfD'
data_inc_2021.loc[(data_inc_2021['city'] == 'Floersheim am Main') & (data_inc_2021['party'] == 'GALF'), 'party'] = 'Grüne Alternative Liste Flörsheim'
data_inc_2021.loc[(data_inc_2021['city'] == 'Floersheim am Main') & (data_inc_2021['party'].isna()), 'party'] = 'Die Freien Bürger (dfb Flörsheim)'
data_inc_2021.loc[(data_inc_2021['city'] == 'Gelnhausen') & (data_inc_2021['party'] == 'BG'), 'party'] = 'Bürger für Gelnhausen (BG)'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Kelsterbach', na=False)) & (data_inc_2021.index == 1509), 'party'] = 'Europäische Union Kelsterbach (E.U.K.)'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Kelsterbach', na=False)) & (data_inc_2021.index >= 1510) & (data_inc_2021.index <= 1512), 'party'] = 'LUB'
data_inc_2021.loc[(data_inc_2021['city'] == 'Nidda') & (data_inc_2021['party'] == 'BL Nidda'), 'party'] = 'Bürger Liste Nidda'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Seligenstadt', na=False)) & (data_inc_2021.index >= 4024) & (data_inc_2021.index <= 4027), 'party'] = 'Grüne'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Seligenstadt', na=False)) & (data_inc_2021.index >= 4028) & (data_inc_2021.index <= 4035), 'party'] = 'FDP'
data_inc_2021.loc[(data_inc_2021['city'].str.contains('Seligenstadt', na=False)) & (data_inc_2021.index >= 4036) & (data_inc_2021.index <= 4039), 'party'] = 'FW'
data_inc_2021.loc[(data_inc_2021['city'] == 'Heusenstamm') & (data_inc_2021['party'].isna()), 'party'] = 'AfD'

data_inc_2021['incumbent'] = 1

data_hessen_2021 = pd.merge(data_hessen_2021, data_inc_2021, on=['city', 'party', 'year', 'last_name', 'first_name'], how='left', validate='m:m')
data_hessen_2021

In [ ]:
data_hessen_2021['muslim'] = 0

def rule_mus(data_hessen_6):
    data_hessen_6.loc[(data_hessen_6['first_name'] == 'Ali') | (data_hessen_6['first_name'] == 'Hüseyin') | 
                    (data_hessen_6['first_name'] == 'Cumali') | (data_hessen_6['first_name'] == 'Nükhet') | 
                    (data_hessen_6['first_name'] == 'Gökhan')
                    | (data_hessen_6['first_name'] == 'Serafettin') | (data_hessen_6['first_name'] == 'Zeynep')
                    | (data_hessen_6['first_name'] == 'Aysel') | (data_hessen_6['first_name'] == 'Gönül') 
                    | (data_hessen_6['first_name'] == 'Ayten') | (data_hessen_6['first_name'] == 'Gülseren') 
                    | (data_hessen_6['first_name'] == 'Ilknur') | (data_hessen_6['first_name'] == 'Kenan')
                    | (data_hessen_6['first_name'] == 'Cetin') | (data_hessen_6['first_name'] == 'Kerem')
                    | (data_hessen_6['first_name'] == 'Oguz') | (data_hessen_6['first_name'] == 'Ilhan')
                    | (data_hessen_6['first_name'] == 'Asya') | (data_hessen_6['first_name'] == 'Tuncay')
                    | (data_hessen_6['first_name'] == 'Hakan') | (data_hessen_6['first_name'] == 'Bülent')
                    | (data_hessen_6['first_name'] == 'Cem') | (data_hessen_6['first_name'] == 'Yasar')
                    | (data_hessen_6['first_name'] == 'Ilknur') | (data_hessen_6['first_name'] == 'Orhan')
                    | (data_hessen_6['first_name'] == 'Resat') | (data_hessen_6['first_name'] == 'Magdi')
                    | (data_hessen_6['first_name'] == 'Teslime') | (data_hessen_6['first_name'] == 'Uygur')
                    | (data_hessen_6['first_name'] == 'Sedef') | (data_hessen_6['first_name'] == 'Ahmet')
                    | (data_hessen_6['first_name'] == 'Mercan') | (data_hessen_6['first_name'] == 'Aziz')
                    | (data_hessen_6['first_name'] == 'Ismail') | (data_hessen_6['first_name'] == 'Ümit')
                    | (data_hessen_6['first_name'] == 'Yasemin') | (data_hessen_6['first_name'] == 'Aynur')
                    | (data_hessen_6['first_name'] == 'Abdurrahman') | (data_hessen_6['first_name'] == 'Süleyman')
                    | (data_hessen_6['first_name'] == 'Mustafa') | (data_hessen_6['first_name'] == 'Senem')
                    | (data_hessen_6['first_name'] == 'Özlem') | (data_hessen_6['first_name'] == 'Taylan')
                    | (data_hessen_6['first_name'] == 'Zeki') | (data_hessen_6['first_name'] == 'Nazile')
                    | (data_hessen_6['first_name'] == 'Ümran') | (data_hessen_6['first_name'] == 'Ferhat')
                    | (data_hessen_6['first_name'] == 'Neslihan') | (data_hessen_6['first_name'] == 'Maksude')
                    | (data_hessen_6['first_name'] == 'Fadime') | (data_hessen_6['first_name'] == 'Güray')
                    | (data_hessen_6['first_name'] == 'Metin') | (data_hessen_6['first_name'] == 'Ramazan')
                    | (data_hessen_6['first_name'] == 'Bediye') | (data_hessen_6['first_name'] == 'Hasim')
                    | (data_hessen_6['first_name'] == 'Bekir') | (data_hessen_6['first_name'] == 'Osman')
                    | (data_hessen_6['first_name'] == 'Gülten') | (data_hessen_6['first_name'] == 'Refiye')
                    | (data_hessen_6['first_name'] == 'Hatun') | (data_hessen_6['first_name'] == 'Arif')
                    | (data_hessen_6['first_name'] == 'Hassan') | (data_hessen_6['first_name'] == 'Celal')
                    | (data_hessen_6['first_name'] == 'Ibrahim') | (data_hessen_6['first_name'] == 'Sevim')
                    | (data_hessen_6['first_name'] == 'Sait') | (data_hessen_6['first_name'] == 'Yetvart')
                    | (data_hessen_6['first_name'] == 'Ugur') | (data_hessen_6['first_name'] == 'Mehmet')
                    | (data_hessen_6['first_name'] == 'Cemal') | (data_hessen_6['first_name'] == 'Tülay')
                    | (data_hessen_6['first_name'] == 'Muhammet') | (data_hessen_6['first_name'] == 'Ayaz')
                    | (data_hessen_6['first_name'] == 'Mahbubur') | (data_hessen_6['first_name'] == 'Durdu')
                    | (data_hessen_6['first_name'] == 'Tugay') | (data_hessen_6['first_name'] == 'Cengiz')
                    | (data_hessen_6['first_name'] == 'Ekrem') | (data_hessen_6['first_name'] == 'Zehra')
                    | (data_hessen_6['first_name'] == 'Hüsamettin') | (data_hessen_6['first_name'] == 'Müslüm')
                    | (data_hessen_6['first_name'] == 'Kevser') | (data_hessen_6['first_name'] == 'Baki')
                    | (data_hessen_6['first_name'] == 'Ikbal') | (data_hessen_6['first_name'] == 'Nejdet')
                    | (data_hessen_6['first_name'] == 'Güllü') | (data_hessen_6['first_name'] == 'Fikret')
                    | (data_hessen_6['first_name'] == 'Handan') | (data_hessen_6['first_name'] == 'Tezcan')
                    | (data_hessen_6['first_name'] == 'Gülhan') | (data_hessen_6['first_name'] == 'Özel')
                    | (data_hessen_6['first_name'] == 'Hemdegül') | (data_hessen_6['first_name'] == 'Nail')
                    | (data_hessen_6['first_name'] == 'Babür') | (data_hessen_6['first_name'] == 'Deniz')
                    | (data_hessen_6['first_name'] == 'Songül') | (data_hessen_6['first_name'] == 'Nebil')
                    | (data_hessen_6['first_name'] == 'Alper') | (data_hessen_6['first_name'] == 'Gülay')
                    | (data_hessen_6['first_name'] == 'Dilan') | (data_hessen_6['first_name'] == 'Nebil')
                    | (data_hessen_6['first_name'] == 'Birgül') | (data_hessen_6['first_name'] == 'Melihat')
                    | (data_hessen_6['first_name'] == 'Devrim') | (data_hessen_6['first_name'] == 'Arzu')
                    | (data_hessen_6['first_name'] == 'Erkan') | (data_hessen_6['first_name'] == 'Nesrin')
                    | (data_hessen_6['first_name'] == 'Ercan') | (data_hessen_6['first_name'] == 'Abdulmesih')
                    | (data_hessen_6['first_name'] == 'Ayhan') | (data_hessen_6['first_name'] == 'Aydin')
                    | (data_hessen_6['first_name'] == 'Kamran') | (data_hessen_6['first_name'] == 'Munib')
                    | (data_hessen_6['first_name'] == 'Hamed') | (data_hessen_6['first_name'] == 'Çigdem')
                    | (data_hessen_6['first_name'] == 'Hülya') | (data_hessen_6['first_name'] == 'Zeynel')
                    | (data_hessen_6['first_name'] == 'Zafer') | (data_hessen_6['first_name'] == 'Dalal')
                    | (data_hessen_6['first_name'] == 'Cafer') | (data_hessen_6['first_name'] == 'Yasin')
                    | (data_hessen_6['first_name'] == 'Batoul') | (data_hessen_6['first_name'] == 'Murat')
                    | (data_hessen_6['first_name'] == 'Halil') | (data_hessen_6['first_name'] == 'Gülcin')
                    | (data_hessen_6['first_name'] == 'Zafar') | (data_hessen_6['first_name'] == 'Sanjar')
                    | (data_hessen_6['first_name'] == 'Tugba') | (data_hessen_6['first_name'] == 'Raziye')
                    | (data_hessen_6['first_name'] == 'Aliye') | (data_hessen_6['first_name'] == 'Niyazi')
                    | (data_hessen_6['first_name'] == 'Kemal') | (data_hessen_6['first_name'] == 'Özgür')
                    | (data_hessen_6['first_name'] == 'Veysel') | (data_hessen_6['first_name'] == 'Khaled')
                    | (data_hessen_6['first_name'] == 'Idris') | (data_hessen_6['first_name'] == 'Yilmaz')
                    | (data_hessen_6['first_name'] == 'Önder') | (data_hessen_6['first_name'] == 'Tülin')
                    | (data_hessen_6['first_name'] == 'Göktürk') | (data_hessen_6['first_name'] == 'Sadik')
                    | (data_hessen_6['first_name'] == 'Selvet') | (data_hessen_6['first_name'] == 'Abdellatif')
                    | (data_hessen_6['first_name'] == 'Hava') | (data_hessen_6['first_name'] == 'Sengül')
                    | (data_hessen_6['first_name'] == 'Erhan') | (data_hessen_6['first_name'] == 'Ergün')
                    | (data_hessen_6['first_name'] == 'Serdar') | (data_hessen_6['first_name'] == 'Yunus')
                    | (data_hessen_6['first_name'] == 'Nurullah') | (data_hessen_6['first_name'] == 'Nida')
                    | (data_hessen_6['first_name'] == 'Nursan') | (data_hessen_6['first_name'] == 'Berkkan')
                    | (data_hessen_6['first_name'] == 'Ertürk') | (data_hessen_6['first_name'] == 'Sakine')
                    | (data_hessen_6['first_name'] == 'Ufuk') | (data_hessen_6['first_name'] == 'Kasim')
                    | (data_hessen_6['first_name'] == 'Lütfi') | (data_hessen_6['first_name'] == 'Emrah')
                    | (data_hessen_6['first_name'] == 'Gürsoy') | (data_hessen_6['first_name'] == 'Yusuf')
                    | (data_hessen_6['first_name'] == 'Maryam') | (data_hessen_6['first_name'] == 'Rabiye')
                    | (data_hessen_6['first_name'] == 'Gülbeyaz') | (data_hessen_6['first_name'] == 'Nezaket')
                    | (data_hessen_6['first_name'] == 'Riza') | (data_hessen_6['first_name'] == 'Görkan')
                    | (data_hessen_6['first_name'] == 'Mohamad') | (data_hessen_6['first_name'] == 'Mohammad')
                    | (data_hessen_6['first_name'] == 'Yavuz') | (data_hessen_6['first_name'] == 'Hachem')
                    | (data_hessen_6['first_name'] == 'Kaan') | (data_hessen_6['first_name'] == 'Erdal') , 'muslim' ] = 1
    
    return data_hessen_6

data_hessen_2021 = rule_mus(data_hessen_2021)
data_hessen_2021

In [ ]:
data_hessen_2021['incumbent'] = data_hessen_2021['incumbent'].fillna(0)

In [ ]:
data_hessen_2021['last_letter'] = data_hessen_2021.last_name.str[-1]
data_hessen_2021['pre_last_letter'] = data_hessen_2021.last_name.str[-2]
data_hessen_2021['third_last_letter'] = data_hessen_2021.last_name.str[-3]
data_hessen_2021['fourth_last_letter'] = data_hessen_2021.last_name.str[-4]
data_hessen_2021['fifth_last_letter'] = data_hessen_2021.last_name.str[-5]
data_hessen_2021['sixth_last_letter'] = data_hessen_2021.last_name.str[-6]
data_hessen_2021['seventh_last_letter'] = data_hessen_2021.last_name.str[-7]
data_hessen_2021['eigth_last_letter'] = data_hessen_2021.last_name.str[-8]

data_hessen_2021['last_letter_first_name'] = data_hessen_2021.first_name.str[-1]
data_hessen_2021['pre_last_letter_first_name'] = data_hessen_2021.first_name.str[-2]
data_hessen_2021['third_last_letter_first_name'] = data_hessen_2021.first_name.str[-3]

In [ ]:
data_hessen_2021['non_muslim_migrant'] = 0

# Greek name structure

def non_m_mig(data_hessen_6):
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'u') & (data_hessen_6['pre_last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'i')
                    & (data_hessen_6['third_last_letter'] == 'd') & (data_hessen_6['fourth_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'i')
                    & (data_hessen_6['third_last_letter'] == 't') & (data_hessen_6['fourth_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'i')
                    & (data_hessen_6['third_last_letter'] == 'd') & (data_hessen_6['fourth_last_letter'] == 'a'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                    & (data_hessen_6['third_last_letter'] == 's') & (data_hessen_6['fourth_last_letter'] == 't')
                    , 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                    & (data_hessen_6['third_last_letter'] == 'l') & (data_hessen_6['fourth_last_letter'] == 'u')
                    , 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                    & (data_hessen_6['third_last_letter'] == 'r'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                    & (data_hessen_6['third_last_letter'] == 'k'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                    & (data_hessen_6['third_last_letter'] == 's'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'o')
                    & (data_hessen_6['third_last_letter'] == 'n'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 's') & (data_hessen_6['pre_last_letter'] == 'a')
                    & (data_hessen_6['third_last_letter'] == 'r'), 'non_muslim_migrant' ] = 1

    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i') & (data_hessen_6['pre_last_letter'] == 'k')
                    & (data_hessen_6['third_last_letter'] == 's'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'y') & (data_hessen_6['pre_last_letter'] == 'k')
                    & (data_hessen_6['third_last_letter'] == 's'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'k') & (data_hessen_6['pre_last_letter'] == 'y')
                    & (data_hessen_6['third_last_letter'] == 'z'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'k') & (data_hessen_6['pre_last_letter'] == 'a')
                    & (data_hessen_6['third_last_letter'] == 'z')
                    , 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'z') & (data_hessen_6['pre_last_letter'] == 'c')
                    & (data_hessen_6['third_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'k') & (data_hessen_6['pre_last_letter'] == 'e')
                    , 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'c') & (data_hessen_6['pre_last_letter'] == 'i')
                    & (data_hessen_6['third_last_letter'] == 'v'), 'non_muslim_migrant' ] = 1

    data_hessen_6.loc[(data_hessen_6['last_letter_first_name'] == 'v') & (data_hessen_6['pre_last_letter_first_name'] == 'a')
                    & (data_hessen_6['third_last_letter_first_name'] == 'l'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter_first_name'] == 'w') & (data_hessen_6['pre_last_letter_first_name'] == 'e')
                    & (data_hessen_6['third_last_letter_first_name'] == 'j'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'y') & (data_hessen_6['pre_last_letter'] == 'n')
                    , 'non_muslim_migrant' ] = 1

    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i') & (data_hessen_6['pre_last_letter'] == 'n')
                    & (data_hessen_6['third_last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'a') & (data_hessen_6['pre_last_letter'] == 'i')
                    , 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i') & (data_hessen_6['pre_last_letter'] == 'n')
                    & (data_hessen_6['third_last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'i'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'e') & (data_hessen_6['pre_last_letter'] == 'n')
                    & (data_hessen_6['third_last_letter'] == 'o'), 'non_muslim_migrant' ] = 1
    data_hessen_6.loc[(data_hessen_6['last_letter'] == 'e') & (data_hessen_6['pre_last_letter'] == 's')
                    & (data_hessen_6['third_last_letter'] == 'e'), 'non_muslim_migrant' ] = 1
    
    return data_hessen_6

data_hessen_2021 = non_m_mig(data_hessen_2021)
data_hessen_2021 = data_hessen_2021.drop(['last_letter_first_name', 'pre_last_letter_first_name', 'third_last_letter_first_name', 
                                'last_letter', 'pre_last_letter', 'third_last_letter', 'fourth_last_letter', 'fifth_last_letter', 'sixth_last_letter', 'seventh_last_letter', 'eigth_last_letter'], axis = 1)

data_hessen_2021.loc[(data_hessen_2021['non_muslim_migrant'] == 1) & (data_hessen_2021['muslim'] == 1), 'non_muslim_migrant'] = 0

In [ ]:
data_hessen_2021['Male'] = data_hessen_2021['Female']*(-1) + 1

In [ ]:
data_hessen = data_hessen_2021.append(data_hessen_hist)

In [ ]:
data_hessen.to_csv(r'C:\Users\mariu\Documents\hessen.csv')

---

In [3]:
import numpy as np
import pandas as pd

In [4]:
data_hessen = pd.read_csv(r'C:\Users\mariu\Documents\hessen.csv', encoding='utf-8', sep=';')
data_hessen.loc[data_hessen['year'] == 2021, 'placelist'] = data_hessen['place_list']
data_hessen.loc[data_hessen['year'] != 2021, 'placelist'] = data_hessen['Spalte1']
data_hessen.drop(['place_list', 'Spalte1'], axis=1, inplace=True)
data_hessen['placelist'] = data_hessen['placelist'].astype(float)
data_hessen.head()

C:\Users\mariu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,4,7,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,doctor,year,incumbent,muslim,non_muslim_migrant,Male,plz,votes,candidates,candidate,placelist
0,Limburg,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,1.0,2021,1,0,0,1,65549,NaN,NaN,NaN,1.0
1,Limburg,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,0.0,2021,1,0,0,1,65549,NaN,NaN,NaN,2.0
2,Limburg,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,0.0,2021,1,0,0,1,65549,NaN,NaN,NaN,3.0
3,Limburg,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,0.0,2021,1,0,0,1,65549,NaN,NaN,NaN,4.0
4,Limburg,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,0.0,2021,1,0,0,1,65549,NaN,NaN,NaN,5.0


In [5]:
import re

for party in ('SPD', 'FDP', 'CDU', 'FW', 'Grüne', ' Linke', 'Piraten', 'Volt', 'PARTEI', 'REP', 'Klimaliste', 'NPD', 'Anarchistische'):
    data_hessen[party] = 0
    data_hessen.loc[data_hessen['party'].str.contains(party, flags=re.IGNORECASE, na=False), party] = 1
data_hessen.head()

,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,doctor,...,FW,Grüne,Linke,Piraten,Volt,PARTEI,REP,Klimaliste,NPD,Anarchistische
0,Limburg,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Limburg,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Limburg,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
3,Limburg,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Limburg,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data_hessen['city'] = data_hessen['city'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('ß', 'ss')
data_hessen.loc[data_hessen['city'] == 'Limburg', 'city'] = 'Limburg a.d.Lahn'
data_hessen.loc[data_hessen['city'] == 'Offenbach', 'city'] = 'Offenbach am Main'
data_hessen.loc[data_hessen['city'] == 'Bad Homburg v. d. Hoehe', 'city'] = 'Bad Homburg v.d.Hoehe'
data_hessen.loc[data_hessen['city'] == 'Offenbach', 'city'] = 'Offenbach am Main'

In [7]:
data_hessen.dropna(subset=['incumbent'], inplace=True)
data_hessen

,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,doctor,...,FW,Grüne,Linke,Piraten,Volt,PARTEI,REP,Klimaliste,NPD,Anarchistische
0,Limburg a.d.Lahn,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Limburg a.d.Lahn,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
2,Limburg a.d.Lahn,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
3,Limburg a.d.Lahn,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
4,Limburg a.d.Lahn,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77333,Zwingenberg,SPD,NaN,Hans,Wiesenthal,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
77334,Zwingenberg,SPD,NaN,Herbert,Woehnl,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
77335,Zwingenberg,SPD,NaN,Lidie,Woehnl,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
77336,Zwingenberg,FDP,NaN,Joachim,Wrona,NaN,NaN,NaN,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data_city_controls = pd.read_csv(r'C:\Users\mariu\Downloads\city_controls.csv', sep=';', encoding='latin1')
data_city_controls['city'] = data_city_controls['city'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('ß', 'ss')
data_hessen = pd.merge(data_hessen, data_city_controls, on=['city', 'year'], how='inner', validate='m:1')
data_hessen.dropna(subset=['tourism_arrivals'], inplace=True)
data_hessen.reset_index(drop=True, inplace=True)
data_hessen.drop(['city_size_industry', 'city_size_work_ha', 'city_size_leisure_ha', 'city_size_green_ha', 'size_mines_ha'], axis=1, inplace=True)
data_hessen.head()

,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,doctor,...,sqm,new_work_buildings,work_sqm,city_size_ha,city_size_living_ha,size_agriculture_ha,size_forest_ha,size_water_ha,tourism_nights,tourism_arrivals
0,Limburg a.d.Lahn,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,1.0,...,"15,1",13.0,"6,4",4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
1,Limburg a.d.Lahn,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,0.0,...,"15,1",13.0,"6,4",4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
2,Limburg a.d.Lahn,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,0.0,...,"15,1",13.0,"6,4",4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
3,Limburg a.d.Lahn,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,0.0,...,"15,1",13.0,"6,4",4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
4,Limburg a.d.Lahn,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,0.0,...,"15,1",13.0,"6,4",4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0


In [9]:
for column in ('work_sqm', 'sqm'):
    data_hessen[column] = data_hessen[column].astype(str).str.replace(',', '.').astype(float)
data_hessen

,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,doctor,...,sqm,new_work_buildings,work_sqm,city_size_ha,city_size_living_ha,size_agriculture_ha,size_forest_ha,size_water_ha,tourism_nights,tourism_arrivals
0,Limburg a.d.Lahn,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,1.0,...,15.1,13.0,6.4,4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
1,Limburg a.d.Lahn,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,0.0,...,15.1,13.0,6.4,4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
2,Limburg a.d.Lahn,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,0.0,...,15.1,13.0,6.4,4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
3,Limburg a.d.Lahn,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,0.0,...,15.1,13.0,6.4,4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
4,Limburg a.d.Lahn,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,0.0,...,15.1,13.0,6.4,4516.0,1239.0,2327.0,247.0,6.0,161008.0,93385.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70895,Zierenberg,SPD,NaN,Angelika,Vialon,NaN,NaN,NaN,NaN,0.0,...,0.7,3.0,1.1,8653.0,181.0,3358.0,4385.0,47.0,4097.0,2506.0
70896,Zierenberg,FDP,NaN,Joachim,Wadsack,NaN,NaN,NaN,NaN,0.0,...,0.7,3.0,1.1,8653.0,181.0,3358.0,4385.0,47.0,4097.0,2506.0
70897,Zierenberg,SPD,NaN,Guenter,Weide,NaN,NaN,NaN,NaN,0.0,...,0.7,3.0,1.1,8653.0,181.0,3358.0,4385.0,47.0,4097.0,2506.0
70898,Zierenberg,CDU,NaN,Juergen,Wenzel,NaN,NaN,NaN,NaN,0.0,...,0.7,3.0,1.1,8653.0,181.0,3358.0,4385.0,47.0,4097.0,2506.0


In [10]:
data_votes = pd.read_csv(r'C:\Users\mariu\Downloads\votes.csv', sep=';', encoding='latin1')
data_hessen = pd.merge(data_hessen, data_votes, on=['year', 'city', 'party', 'code'], how='left', validate='m:1')

In [11]:
data_party = pd.read_csv(r'C:\Users\mariu\Downloads\party_controls.csv', sep=';', encoding='utf-8')
data_hessen = pd.merge(data_hessen, data_party, on=['year', 'party'], how='left', validate='m:1')
data_hessen.loc[data_hessen['diff_poll_to_last_result'].isna(), 'diff_poll_to_last_result'] = 0
data_hessen.loc[data_hessen['avg_poll_percent'].isna(), 'avg_poll_percent'] = 0
data_hessen.loc[data_hessen['rile'].isna(), 'rile'] = 0

In [12]:
data_year = pd.read_csv(r'C:\Users\mariu\Downloads\controls_year.csv', sep=';', encoding='utf-8')
data_year.drop('Unnamed: 10', axis=1, inplace=True)
data_hessen = pd.merge(data_hessen, data_year, on=['year'], how='inner', validate='m:1')

In [13]:
data_hessen['votes_party'] = data_hessen['votes_party'].astype(float)
data_hessen['votes_total'] = data_hessen['votes_total'].astype(float)
data_hessen['share_votes'] = data_hessen['votes_party']/data_hessen['votes_total']
data_hessen.loc[data_hessen['share_votes'].isna(), 'share_votes'] = 0

In [14]:
list_controls_toshare = [col for col in data_hessen if col.startswith('unemployed') or col.startswith('female') or col.startswith('male')]

for name in range(len(list_controls_toshare)):
    name_column = list_controls_toshare[name] + '_share'
    data_hessen[name_column] = data_hessen[list_controls_toshare[name]] / data_hessen['pop_total']
    
data_hessen

,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,doctor,...,female_25_30_share,female_30_35_share,female_35_40_share,female_40_45_share,female_45_50_share,female_50_55_share,female_55_60_share,female_60_65_share,female_65_75_share,female_75_more_share
0,Limburg a.d.Lahn,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,1.0,...,0.029115,0.031424,0.029538,0.028383,0.031199,0.039337,0.039703,0.034127,0.056344,0.067973
1,Limburg a.d.Lahn,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,0.0,...,0.029115,0.031424,0.029538,0.028383,0.031199,0.039337,0.039703,0.034127,0.056344,0.067973
2,Limburg a.d.Lahn,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,0.0,...,0.029115,0.031424,0.029538,0.028383,0.031199,0.039337,0.039703,0.034127,0.056344,0.067973
3,Limburg a.d.Lahn,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,0.0,...,0.029115,0.031424,0.029538,0.028383,0.031199,0.039337,0.039703,0.034127,0.056344,0.067973
4,Limburg a.d.Lahn,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,0.0,...,0.029115,0.031424,0.029538,0.028383,0.031199,0.039337,0.039703,0.034127,0.056344,0.067973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70895,Zierenberg,SPD,NaN,Angelika,Vialon,NaN,NaN,NaN,NaN,0.0,...,0.020995,0.020995,0.034231,0.041229,0.044120,0.037578,0.036057,0.034535,0.060551,0.074091
70896,Zierenberg,FDP,NaN,Joachim,Wadsack,NaN,NaN,NaN,NaN,0.0,...,0.020995,0.020995,0.034231,0.041229,0.044120,0.037578,0.036057,0.034535,0.060551,0.074091
70897,Zierenberg,SPD,NaN,Guenter,Weide,NaN,NaN,NaN,NaN,0.0,...,0.020995,0.020995,0.034231,0.041229,0.044120,0.037578,0.036057,0.034535,0.060551,0.074091
70898,Zierenberg,CDU,NaN,Juergen,Wenzel,NaN,NaN,NaN,NaN,0.0,...,0.020995,0.020995,0.034231,0.041229,0.044120,0.037578,0.036057,0.034535,0.060551,0.074091


In [15]:
data_city_controls.columns
list_pop = ['tourism_nights', 'tourism_arrivals', 'pop_male', 'pop_female', 'pop_0_3', 'pop_3_6', 'pop_6_10', 'pop_10_15', 'pop_15_18', 'pop_18_20', 'pop_20_25', 'pop_25_30', 'pop_30_35', 'pop_35_40', 'pop_40_45', 'pop_45_50', 'pop_50_55', 'pop_55_60', 'pop_60_65', 'pop_65_75', 'pop_75_more']

for i in range(len(list_pop)):
    name = list_pop[i] + '_share'
    data_hessen[name] = data_hessen[list_pop[i]]/data_hessen['pop_total']
    
data_hessen

,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,doctor,...,pop_25_30_share,pop_30_35_share,pop_35_40_share,pop_40_45_share,pop_45_50_share,pop_50_55_share,pop_55_60_share,pop_60_65_share,pop_65_75_share,pop_75_more_share
0,Limburg a.d.Lahn,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,1.0,...,0.062539,0.066368,0.062229,0.058315,0.062595,0.077744,0.077857,0.066509,0.104607,0.114321
1,Limburg a.d.Lahn,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,0.0,...,0.062539,0.066368,0.062229,0.058315,0.062595,0.077744,0.077857,0.066509,0.104607,0.114321
2,Limburg a.d.Lahn,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,0.0,...,0.062539,0.066368,0.062229,0.058315,0.062595,0.077744,0.077857,0.066509,0.104607,0.114321
3,Limburg a.d.Lahn,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,0.0,...,0.062539,0.066368,0.062229,0.058315,0.062595,0.077744,0.077857,0.066509,0.104607,0.114321
4,Limburg a.d.Lahn,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,0.0,...,0.062539,0.066368,0.062229,0.058315,0.062595,0.077744,0.077857,0.066509,0.104607,0.114321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70895,Zierenberg,SPD,NaN,Angelika,Vialon,NaN,NaN,NaN,NaN,0.0,...,0.045489,0.044120,0.061464,0.084741,0.090370,0.078807,0.069679,0.069375,0.120493,0.113038
70896,Zierenberg,FDP,NaN,Joachim,Wadsack,NaN,NaN,NaN,NaN,0.0,...,0.045489,0.044120,0.061464,0.084741,0.090370,0.078807,0.069679,0.069375,0.120493,0.113038
70897,Zierenberg,SPD,NaN,Guenter,Weide,NaN,NaN,NaN,NaN,0.0,...,0.045489,0.044120,0.061464,0.084741,0.090370,0.078807,0.069679,0.069375,0.120493,0.113038
70898,Zierenberg,CDU,NaN,Juergen,Wenzel,NaN,NaN,NaN,NaN,0.0,...,0.045489,0.044120,0.061464,0.084741,0.090370,0.078807,0.069679,0.069375,0.120493,0.113038


In [16]:
data_hessen['share_living_ha'] = data_hessen['city_size_living_ha']/data_hessen['city_size_ha']
data_hessen['share_agriculture'] = data_hessen['size_agriculture_ha']/data_hessen['city_size_ha']
data_hessen['share_forest'] = data_hessen['size_forest_ha']/data_hessen['city_size_ha']
data_hessen['share_water'] = data_hessen['size_water_ha']/data_hessen['city_size_ha']

data_hessen['density_city'] = data_hessen['city_size_ha']/data_hessen['pop_total']
data_hessen['uni'] = 0
list_uni = ['Giessen', 'Frankfurt', 'Marburg', 'Kassel', 'Darmstadt']
data_hessen.loc[data_hessen['city'].isin(list_uni), 'uni'] = 1
data_hessen['prison'] = 0
list_prison = ['Frankfurt', 'Kassel', 'Butzbach', 'Darmstadt', 'Dieburg', 'Fulda', 'Giessen', 'Limburg a.d.Lahn', 'Rockenberg', 'Schwalmstadt', 'Weiterstadt', 'Wiesbaden']
data_hessen.loc[data_hessen['city'].isin(list_prison), 'prison'] = 1

In [ ]:
data_hessen.loc[data_hessen['doctor'].isna(), 'doctor'] = 0 

---

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import time
from sklearn.ensemble import GradientBoostingRegressor # Model 3
from sklearn.ensemble import RandomForestRegressor # Model 2
from sklearn.linear_model import Lasso, PoissonRegressor # Model 1
from sklearn.model_selection import GridSearchCV # Hypertuning of parameters
from sklearn.svm import LinearSVR # Model 4
from sklearn.ensemble import VotingRegressor # Ensemble estimator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

rng = np.random.RandomState(0)
scaler = MinMaxScaler()

In [ ]:
data_train = data_hessen[data_hessen['year'] != 2021]
data_test = data_hessen[data_hessen['year'] == 2021]

In [ ]:
X_names = ['doctor', 'incumbent', 'muslim', 'non_muslim_migrant', 'Male', 'SPD', 'FDP', 'CDU', 'FW', 'Grüne', ' Linke', 'Piraten', 'Volt', 'PARTEI', 'REP', 'Klimaliste', 'NPD', 'Anarchistische', 'share_votes',
           'unemployed_share', 'unemployed_foreign_share', 'unemployed_young_share', 'unemployed_old_share', 'unemployed_longterm_share', 'male_0_3_share', 'male_3_6_share', 'male_6_10_share', 'male_10_15_share', 'male_15_18_share',
           'male_18_20_share', 'male_20_25_share', 'male_25_30_share', 'male_30_35_share', 'male_35_40_share', 'male_40_45_share', 'male_45_50_share', 'male_50_55_share', 'male_55_60_share', 'male_60_65_share', 'male_65_75_share',
           'male_75_more_share', 'female_0_3_share', 'female_3_6_share', 'female_6_10_share', 'female_10_15_share', 'female_15_18_share', 'female_18_20_share', 'female_20_25_share', 'female_25_30_share', 'female_30_35_share', 
           'female_35_40_share', 'female_40_45_share', 'female_45_50_share', 'female_50_55_share', 'female_55_60_share', 'female_60_65_share', 'female_65_75_share', 'female_75_more_share', 'tourism_nights_share', 'tourism_arrivals_share',
           'pop_male_share', 'pop_female_share', 'pop_0_3_share', 'pop_3_6_share', 'pop_6_10_share', 'pop_10_15_share', 'pop_15_18_share', 'pop_18_20_share', 'pop_20_25_share', 'pop_25_30_share', 'pop_30_35_share', 'pop_35_40_share',
           'pop_40_45_share', 'pop_45_50_share', 'pop_50_55_share', 'pop_55_60_share', 'pop_60_65_share', 'pop_65_75_share', 'pop_75_more_share', 'share_living_ha', 'share_agriculture', 'share_forest', 'share_water', 'uni', 'prison']

to_scale = ['placelist', 'pop_total', 'avg_poll_percent', 'diff_poll_to_last_result', 'rile', 'diff_avg_tax', 'diff_avg_security', 'diff_avg_religion', 'diff_avg_terrorism', 'diff_avg_fridays_ff', 'diff_avg_health', 'diff_avg_crisis',
            'diff_avg_minwage', 'diff_avg_poverty', 'density_city'] # scale 0-1

In [ ]:
data_train.dropna(subset=X_names, inplace=True)
data_train.dropna(subset=to_scale, inplace=True)
data_test.dropna(subset=['placelist'], inplace=True)

In [ ]:
X_train_step = data_train[X_names]
X_test_step = data_test[X_names]

X_train_scale = data_train[to_scale]
X_test_scale = data_test[to_scale]

X_train_scaled = X_train_scale.copy()
X_test_scaled = X_test_scale.copy()

X_train_scaled_2 = MinMaxScaler().fit_transform(X_train_scale)
X_test_scaled_2 = MinMaxScaler().fit(X_train_scale).transform(X_test_scale)

X_train_scaled[to_scale] = X_train_scaled_2
X_test_scaled[to_scale] = X_test_scaled_2

X_train = pd.concat([X_train_step, X_train_scaled],axis=1).reset_index(drop=True)
X_test = pd.concat([X_test_step, X_test_scaled],axis=1).reset_index(drop=True)

y = data_train['votes'].reset_index(drop=True)

X_train.head(), X_test.head(), y.head()

In [ ]:
X_test.columns[X_test.isna().any()].tolist(), X_train.columns[X_train.isna().any()].tolist()

In [ ]:
np.isfinite(X_test).all(), np.isfinite(X_train).all()

In [ ]:
param_test1 = {'n_estimators':range(20,200,40), 'max_depth':range(5,18,4)}
param_test2 = {'min_samples_split':range(200,1001,100), 'min_samples_leaf': range(10, 200, 30)}

t1 = time.time()
gradient_boosting_cv = GridSearchCV(estimator = GradientBoostingRegressor(min_samples_leaf=100, min_samples_split=400, random_state=rng), param_grid = param_test1, cv=10, error_score='raise', verbose = 3, n_jobs = -1)
t_grad_cv = time.time() - t1

t2 = time.time()
gradient_boosting_cv.fit(X_train, y)
t_grad_reg = time.time() - t2

gradient_boosting_cv.best_params_, gradient_boosting_cv.best_score_, t1, t2, gradient_boosting_cv.score(X_train,y)

In [ ]:
grad_boost = GradientBoostingRegressor(random_state=1, min_samples_leaf = 100, 
                                    n_estimators=60, loss = 'ls', learning_rate = 0.1, max_features = 'sqrt',
                                    criterion = 'mse', verbose = 0, max_depth = 9, min_samples_split = 400)

grad_boost.fit(X_train, y)
grad_boost.score(X_train, y), grad_boost.score(X_train, y)

In [ ]:
param_grid_1 = {
    #'max_depth': range(10,100,10),
    #'min_samples_leaf': [4, 8, 12, 16, 20],
    #'min_samples_split': [10,20,30,40,50],
    'n_estimators': range(10,100,10),
}

random_forest = GridSearchCV(estimator = RandomForestRegressor(max_depth= 30, min_samples_leaf = 20, min_samples_split=50)
                             , param_grid = param_grid_1, cv = 10, verbose = 5, n_jobs = -1)
t2 = time.time()
random_forest.fit(X_train, y)
t_random_forest = time.time() - t2


random_forest.score(X_train,y), random_forest.best_params_

In [ ]:
rf = RandomForestRegressor(max_depth= 30, min_samples_leaf= 4, min_samples_split= 50, n_estimators = 60)
rf.fit(X_train, y), rf.score(X_train,y)

In [ ]:
Cs = [1, 2, 5, 10]
param_grid_svr = {'C': Cs, 'max_iter': range(1000, 10000, 2000)}

svr_cv = GridSearchCV(estimator = LinearSVR(max_iter=5000), param_grid = param_grid_svr, cv=5, n_jobs = -1, verbose = 1)
svr_cv.fit(X_train, y)
svr_cv.best_params_, svr_cv.score(X_train, y)

In [ ]:
svr = LinearSVR(C=5, max_iter = 5000)
svr.fit(X_train, y), svr.score(X_train, y)

In [ ]:
param_grid_1 = {
    'alpha': np.logspace(0,3,10)
}

lasso_cv = GridSearchCV(estimator = Lasso(max_iter=10000), param_grid = param_grid_1, cv = 10, n_jobs= -1, verbose = 1)
lasso_cv.fit(X_train, y)
lasso_cv.best_params_, lasso_cv.score(X_train, y)

In [ ]:
lasso = Lasso(alpha = 2.154434690031884)
lasso.fit(X_train, y), lasso.score(X_train, y)

In [ ]:
ensemble_regression = VotingRegressor([('random_forest', rf), ('gradient_boosting', grad_boost), ('SVR', svr),
                                       ('Lasso', lasso)])

param_ensemble = {'weights': [(1, 1, 2, 1), (1, 2, 1, 1), (2, 1, 1, 1), (1, 1, 1, 2), (2, 2, 1, 1), (2, 1, 2, 1), (2, 1, 1, 2), (1, 2, 2, 1), (1, 2, 1, 2), (1, 1, 2, 2), (2, 2, 2, 1), (2, 2, 1, 2), (1, 2, 2, 2), (1, 1, 1, 1),
                              (1, 1, 1, 0), (1, 1, 0, 1), (1, 0, 1, 1), (0, 1, 1, 1), (2, 2, 1, 0), (2, 2, 0, 1), (2, 1, 0, 2), (2, 0, 1, 2), (1, 0, 2, 2), (0, 1, 2, 2), (0, 2, 1, 2), (1, 2, 0, 2), (2, 1, 2, 0), (2, 0, 2, 1), (1, 2, 2, 0), 
                              (0, 2, 2, 1), (2, 1, 1, 0), (2, 1, 0, 1), (2, 0, 1, 1), (2, 0, 1, 0), (1, 2, 1, 0), (1, 2, 0, 1), (0, 2, 1, 0), (0, 2, 1, 1), (0, 1, 2, 1), (0, 1, 2, 1), (1, 0, 2, 1), (1, 1, 2, 0), (1, 0, 2, 0), (0, 1, 2, 0),
                              (0, 0, 1, 2), (0, 1, 0, 2), (1, 0, 0, 2), (1, 1, 0, 2), (1, 0, 1, 2), (0, 1, 1, 2)]}

ensemble_cv = GridSearchCV(estimator = ensemble_regression, param_grid = param_ensemble , cv=5, n_jobs = 4, verbose = 2)
ensemble_cv.fit(X_train, y)

ensemble_cv.best_params_, ensemble_cv.score(X_train, y)

In [ ]:
voting_reg = VotingRegressor([('random_forest', rf), ('gradient_boosting', grad_boost), ('SVR', svr),
                             ('Lasso', lasso)], weights = (2,1,0,2))
voting_reg.fit(X_train, y), voting_reg.score(X_train, y)

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt

model_nn = Sequential()
model_nn.add(Dense(100, input_dim=100, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(100, activation='relu'))
model_nn.add(Dense(100, activation='relu'))
model_nn.add(Dense(1, activation='linear'))
model_nn.summary()
model_nn.compile(loss='mse', optimizer='adam', metrics=['mse'])

history = model_nn.fit(X_train, y, epochs=75, batch_size=32,  verbose=8, validation_split=0.2, callbacks=[tfdocs.modeling.EpochDots()])
predictions_nn = model_nn.predict(X_train, verbose=4)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "mse")


In [ ]:
from sklearn.metrics import r2_score
r2_score(y, predictions_nn)

In [ ]:
data_pred = data_test
data_pred['grad_boost_pred'] =grad_boost.predict(X_test)
data_pred['voting_pred'] = voting_reg.predict(X_test)
data_pred['rf_pred'] = rf.predict(X_test)
data_pred['neural_net_pred'] = model_nn.predict(X_test)
data_pred

In [ ]:
data_pred['average'] = (data_pred['grad_boost_pred']*data_pred['voting_pred']*data_pred['rf_pred']*data_pred['neural_net_pred'])**(1/4)
data_pred_2 = pd.DataFrame()
for city in data_pred['city'].unique():
    data_city = data_pred[data_pred['city'] == city]
    data_city['std'] = data_city['average'].std()
    data_pred_2 = data_pred_2.append(data_city)

In [ ]:
data_pred_2['lower_confident'] = data_pred_2['average'] - 1.645*data_pred_2['std']
data_pred_2['upper_confident'] = data_pred_2['average'] + 1.645*data_pred_2['std']
data_pred_2

In [ ]:
data_pred_2.to_csv(r'C:\Users\mariu\Documents\hessen_pred.csv')

---

In [128]:
import pandas as pd
data_pred_2 = pd.read_csv(r'C:\Users\mariu\Documents\hessen_pred.csv')
data_pred_2.head()

C:\Users\mariu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,city,party,sex,first_name,last_name,job,birth_year,birth_place,address,...,uni,prison,grad_boost_pred,voting_pred,rf_pred,neural_net_pred,average,std,lower_confident,upper_confident
0,0,Limburg a.d.Lahn,CDU,Herr,Christopher,Dietz,Historiker,1975.0,in Limburg,NaN,...,0,1,5630.910502,4060.166590,3700.878592,4256.9106,4356.427216,883.65442,2902.815696,5810.038736
1,1,Limburg a.d.Lahn,CDU,Herr,Stefan,Muth,Architekt,1976.0,in Limburg,NaN,...,0,1,5497.290367,3949.020710,3546.157410,3579.7700,4074.391959,883.65442,2620.780439,5528.003480
2,2,Limburg a.d.Lahn,CDU,Herr,Richard,Eisenbach,Steuerberater,1971.0,in Limburg,NaN,...,0,1,4885.094503,3749.980654,2686.136805,3461.2869,3612.575479,883.65442,2158.963958,5066.186999
3,3,Limburg a.d.Lahn,CDU,Herr,Manuel,Barfuss,Ofensetzermeister,1980.0,in Limburg,NaN,...,0,1,4460.188576,3608.000525,2664.469138,3348.7607,3461.614145,883.65442,2008.002624,4915.225665
4,4,Limburg a.d.Lahn,CDU,Herr,Michael,Stock,Geschäftsführer,1973.0,in Limburg,NaN,...,0,1,4438.256056,3580.097596,2694.822982,3239.6980,3431.906898,883.65442,1978.295377,4885.518418


In [251]:
len(data_pred_2)

9714

In [129]:
votes_list = data_pred_2.groupby(['city', 'party'], as_index=False).sum()[['city', 'party', 'average', 'grad_boost_pred', 'voting_pred', 'rf_pred',
                                                                           'neural_net_pred', 'lower_confident', 'upper_confident']]
votes_list_2 = pd.DataFrame()

for count in range(1,41):
    
    votes_list['divided_by'] = count
    votes_list_2 = pd.concat([votes_list_2, votes_list])


In [ ]:
votes_list_2['rank_votes'] = votes_list_2['average'] / votes_list_2['divided_by']
votes_list_2['rank_votes_gb'] = votes_list_2['grad_boost_pred'] / votes_list_2['divided_by']
votes_list_2['rank_votes_vp'] = votes_list_2['voting_pred'] / votes_list_2['divided_by']
votes_list_2['rank_votes_rf'] = votes_list_2['rf_pred'] / votes_list_2['divided_by']
votes_list_2['rank_votes_nn'] = votes_list_2['neural_net_pred'] / votes_list_2['divided_by']
votes_list_2.reset_index(drop=True, inplace=True)
votes_list_2

In [ ]:
data_city_controls = pd.read_csv(r'C:\Users\mariu\Downloads\city_controls.csv', sep=';', encoding='latin1')
data_city_controls = data_city_controls[data_city_controls['year'] == 2021]
data_city_controls = data_city_controls[['pop_total', 'city']]
data_city_controls['city'] = data_city_controls['city'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('ß', 'ss')
votes_list_2 = pd.merge(data_city_controls, votes_list_2, on='city', how='right')

votes_list_2['seats_total'] = 37
votes_list_2.loc[(votes_list_2['pop_total'] >= 25001) & (votes_list_2['pop_total'] <= 50000), 'seats_total'] = 45
votes_list_2.loc[(votes_list_2['pop_total'] >= 50001) & (votes_list_2['pop_total'] <= 100000), 'seats_total'] = 59
votes_list_2.loc[(votes_list_2['pop_total'] >= 100001) & (votes_list_2['pop_total'] <= 250000), 'seats_total'] = 71
votes_list_2.loc[(votes_list_2['pop_total'] >= 250001) & (votes_list_2['pop_total'] <= 500000), 'seats_total'] = 81
votes_list_2.loc[(votes_list_2['pop_total'] >= 500001) & (votes_list_2['pop_total'] <= 1000000), 'seats_total'] = 91
votes_list_2

In [ ]:
data_pred_sorted_1 = pd.DataFrame()

#for estimator in ('rank_votes', 'rank_votes_gb', 'rank_votes_lasso', 'rank_votes_rf', 'rank_votes_vp', 'rank_votes_nn'):
    
for city in votes_list_2['city'].unique():
        
    data_city = votes_list_2[votes_list_2['city'] == city]
    data_city = data_city.sort_values(by='rank_votes', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_pred_sorted_1 = data_pred_sorted_1.append(data_city)
data_pred_sorted_1.rename({'divided_by': 'seats_average'}, axis=1, inplace=True)
data_pred_sorted_1.reset_index(inplace=True)
    
data_pred_sorted_2 = pd.DataFrame()  
for city in votes_list_2['city'].unique():
        
    data_city = votes_list_2[votes_list_2['city'] == city]
    data_city = data_city.sort_values(by='rank_votes_gb', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_pred_sorted_2 = data_pred_sorted_2.append(data_city)
data_pred_sorted_2.rename({'divided_by': 'seats_gb'}, axis=1, inplace=True)
data_pred_sorted_2.reset_index(inplace=True)

data_pred_sorted_3 = pd.DataFrame()
for city in votes_list_2['city'].unique():
        
    data_city = votes_list_2[votes_list_2['city'] == city]
    data_city = data_city.sort_values(by='rank_votes_vp', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_pred_sorted_3 = data_pred_sorted_3.append(data_city)
data_pred_sorted_3.rename({'divided_by': 'seats_vp'}, axis=1, inplace=True)
data_pred_sorted_3.reset_index(inplace=True)
    
data_pred_sorted_4 = pd.DataFrame()
for city in votes_list_2['city'].unique():
        
    data_city = votes_list_2[votes_list_2['city'] == city]
    data_city = data_city.sort_values(by='rank_votes_nn', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_pred_sorted_4 = data_pred_sorted_4.append(data_city)
data_pred_sorted_4.rename({'divided_by': 'seats_nn'}, axis=1, inplace=True)
data_pred_sorted_4.reset_index(inplace=True)


In [ ]:
data_pred_sorted_2[data_pred_sorted_2['city'] == 'Wiesbaden'].tail(10)

In [ ]:
data_pred_sorted_2 = data_pred_sorted_2.sort_values(by=['city', 'party', 'seats_total', 'seats_gb'])
#data_pred_sorted.rename({'divided_by': 'seats'}, axis=1, inplace=True)

In [ ]:
data_pred_sorted_2

In [ ]:
data_pred_sorted_2.to_csv(r'C:\Users\mariu\Documents\hessen_pred_graph.csv')

In [ ]:
data_pred_sorted_2['city'].unique()

In [ ]:
cdu_tot = 0
tot = 0
for city in data_pred_2['city'].unique()[17:]:
    if (data_pred_2[data_pred_2['city'] == city]['party'].str.contains('AfD').any()):
        print(city)
        cdu_tot += max(data_pred_sorted_2[(data_pred_sorted_2['city'] == city) & (data_pred_sorted_2['party'].str.contains('AfD'))]['seats_gb'])
        tot += max(data_pred_sorted_2[(data_pred_sorted_2['city'] == city)]['seats_total'].unique())
        print(tot, cdu_tot)
        
    else:
        continue

In [ ]:
data_pred_sorted[data_pred_sorted['party'].str.contains('Bürger')]['party'].unique()

In [ ]:
x_article = data_train[data_train['city'].isin(data_pred_sorted_2['city'].unique())]
x_article_2 = data_pred_sorted_2[data_pred_sorted_2['city'].isin(data_train['city'].unique())]


In [ ]:
len(x_article[]['party'].unique()), len(x_article_2['party'].unique()), len(x_article['city'].unique())

In [216]:
votes_list_conf = votes_list_2[['party', 'city', 'grad_boost_pred', 'lower_confident', 'upper_confident', 'divided_by']]
votes_list_conf['reduction'] = votes_list_conf['grad_boost_pred'] - votes_list_conf['lower_confident']
votes_list_conf['increase'] = votes_list_conf['upper_confident'] - votes_list_conf['grad_boost_pred']
votes_list_conf

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,party,city,grad_boost_pred,lower_confident,upper_confident,divided_by,reduction,increase
0,CDU,Babenhausen,68282.733383,36893.750457,9.600580e+04,1,31388.982926,27723.068862
1,FDP,Babenhausen,29141.278808,9718.979252,4.486669e+04,1,19422.299556,15725.406912
2,FW,Babenhausen,24618.140019,9787.626483,3.375197e+04,1,14830.513536,9133.831783
3,Grüne,Babenhausen,33323.700806,24701.637396,4.866598e+04,1,8622.063410,15342.281910
4,SPD,Babenhausen,49068.273308,16950.485212,6.487918e+04,1,32117.788096,15810.902542
...,...,...,...,...,...,...,...,...
333,Liberal-Konservative Reformer,Wiesbaden,100925.850610,-81230.973441,3.299421e+05,40,182156.824052,229016.227067
334,PARTEI,Wiesbaden,34022.872206,-40842.872508,1.099206e+05,40,74865.744714,75897.707363
335,SPD,Wiesbaden,521063.461413,229657.804135,1.339825e+06,40,291405.657278,818761.580743
336,Unabhängige Liste Wiesbaden,Wiesbaden,123659.133520,-113904.486456,4.206205e+05,40,237563.619976,296961.346478


In [249]:
votes_list_conf[votes_list_conf['city'] == 'Wiesbaden']

,party,city,grad_boost_pred,lower_confident,upper_confident,divided_by,reduction,increase,Good_night_FDP,Good_night_SPD,...,Good_night_CDU_divided,Good_night_Grüne_divided,Good_night_Die Linke_divided,Good_night_AfD_divided,Bad_night_FDP_divided,Bad_night_SPD_divided,Bad_night_CDU_divided,Bad_night_Grüne_divided,Bad_night_Die Linke_divided,Bad_night_AfD_divided
326,AfD,Wiesbaden,125049.368072,11192.586558,3.949541e+05,1,113856.781514,2.699047e+05,1.166891e+05,9.229890e+04,...,1.250494e+05,125049.368072,118213.853070,394954.100936,174532.192083,154189.933800,129385.963176,129142.148705,96239.304932,11192.586558
327,Bürgerliste Wiesbaden,Wiesbaden,96399.626707,-72882.181922,3.108793e+05,1,169281.808630,2.144797e+05,9.639963e+04,9.639963e+04,...,9.639963e+04,96399.626707,96399.626707,96399.626707,96399.626707,96399.626707,96399.626707,96399.626707,96399.626707,96399.626707
328,CDU,Wiesbaden,502320.189424,483465.428103,1.730690e+06,1,18854.761321,1.228370e+06,1.679100e+05,3.631307e+05,...,1.730690e+06,256889.685754,474978.129417,413251.627579,510133.266899,540202.924870,483465.428103,584175.802079,496254.912973,502320.189424
329,Die Linke,Wiesbaden,157091.036405,5459.125143,4.988668e+05,1,151631.911263,3.417758e+05,1.487308e+05,1.489034e+05,...,1.325236e+05,113264.160750,498866.786486,140896.752434,159695.395564,180403.488988,157468.131632,174826.419147,5459.125143,151398.197330
330,FDP,Wiesbaden,237630.296010,-22805.619835,1.073656e+06,1,260435.915845,8.360256e+05,1.073656e+06,2.212551e+05,...,5.337477e+04,211334.170616,234212.538509,221436.012038,-22805.619835,255114.635446,240458.510208,243087.336853,234597.657784,234214.592564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,Liberal-Konservative Reformer,Wiesbaden,100925.850610,-81230.973441,3.299421e+05,40,182156.824052,2.290162e+05,1.009259e+05,1.009259e+05,...,2.523146e+03,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265
334,PARTEI,Wiesbaden,34022.872206,-40842.872508,1.099206e+05,40,74865.744714,7.589771e+04,3.402287e+04,3.402287e+04,...,8.505718e+02,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805
335,SPD,Wiesbaden,521063.461413,229657.804135,1.339825e+06,40,291405.657278,8.187616e+05,3.956596e+05,1.339825e+06,...,4.427995e+03,4918.614539,10634.156285,12149.396154,14849.637946,5741.445103,13054.868677,13845.142662,12761.230691,12941.193949
336,Unabhängige Liste Wiesbaden,Wiesbaden,123659.133520,-113904.486456,4.206205e+05,40,237563.619976,2.969613e+05,1.236591e+05,1.236591e+05,...,3.091478e+03,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338


In [217]:
for party in ('FDP', 'SPD', 'Grüne', 'Die Linke', 'CDU', 'AfD'):
    name = 'Good_night_' + party
    votes_list_conf[name] = votes_list_conf['grad_boost_pred']
    votes_list_conf.loc[(votes_list_conf['party'] == party), name] = votes_list_conf[votes_list_conf['party'] == party]['upper_confident']
    
    for city in votes_list_conf['city'].unique():
        
        #FDP
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='CDU') & (votes_list_conf['city']==city), 'Good_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='CDU')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['increase'].values * 0.4
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Good_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['increase'].values * 0.15
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Good_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['increase'].values * 0.04
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Good_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['increase'].values * 0.01
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Good_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['increase'].values * 0.01

        #SPD
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='CDU') & (votes_list_conf['city']==city), 'Good_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='CDU')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['increase'].values * 0.17
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Good_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['increase'].values * 0.02
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Good_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['increase'].values * 0.26
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Good_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['increase'].values * 0.04
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Good_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['increase'].values * 0.01

        #CDU
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Good_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['increase'].values * 0.28
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Good_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['increase'].values * 0.15
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Good_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['increase'].values * 0.05
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Good_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['increase'].values * 0.0
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Good_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['increase'].values * 0.02

        #Grüne
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Good_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['increase'].values * 0.37
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Good_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['increase'].values * 0.03
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('CDU')) & (votes_list_conf['city']==city), 'Good_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('CDU'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['increase'].values * 0.28
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Good_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['increase'].values * 0.0
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Linke')) & (votes_list_conf['city']==city), 'Good_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Linke'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['increase'].values * 0.05

        #Linke
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Good_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['increase'].values * 0.28
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Good_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['increase'].values * 0.01
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Good_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['increase'].values * 0.19
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Good_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['increase'].values * 0.02
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('CDU')) & (votes_list_conf['city']==city), 'Good_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('CDU'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['increase'].values * 0.08

        #AfD
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Good_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['increase'].values * 0.13
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Good_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['increase'].values * 0.06
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Good_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['increase'].values * 0.01
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('CDU')) & (votes_list_conf['city']==city), 'Good_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('CDU'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['increase'].values * 0.33
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Good_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['increase'].values * 0.06

    name_2 = 'Bad_night_' + party
    votes_list_conf[name_2] = votes_list_conf['grad_boost_pred']
    votes_list_conf.loc[(votes_list_conf['party'] == party), name_2] = votes_list_conf[votes_list_conf['party'] == party]['lower_confident']
    
    for city in votes_list_conf['city'].unique():
        
        #FDP
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='CDU') & (votes_list_conf['city']==city), 'Bad_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='CDU')]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['reduction'].values * 0.03
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Bad_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['reduction'].values * 0.28
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Bad_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['reduction'].values * 0.1
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Bad_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['reduction'].values * 0.19
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Bad_night_FDP'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='FDP')]['reduction'].values * 0.01
     
        #SPD
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='CDU') & (votes_list_conf['city']==city), 'Bad_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='CDU')]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['reduction'].values * 0.13
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Bad_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['reduction'].values * 0.06
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Bad_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['reduction'].values * 0.36
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Bad_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['reduction'].values * 0.1
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('SPD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Bad_night_SPD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='SPD')]['reduction'].values * 0.08
    
        #CDU
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Bad_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['reduction'].values * 0.06
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Bad_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['reduction'].values * 0.15
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Bad_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['reduction'].values * 0.26
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Bad_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['reduction'].values * 0.23
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('CDU').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Bad_night_CDU'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values + votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='CDU')]['reduction'].values * 0.02
    
        #Grüne
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Bad_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['reduction'].values * 0.24
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Bad_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['reduction'].values * 0.04
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('CDU')) & (votes_list_conf['city']==city), 'Bad_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('CDU'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['reduction'].values * 0.6
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Bad_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['reduction'].values * 0.03
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Bad_night_Grüne'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Grüne'))]['reduction'].values * 0.13

        #Linke
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Bad_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['reduction'].values * 0.07
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Bad_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['reduction'].values * 0.02
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Bad_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['reduction'].values * 0.22
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('AfD')) & (votes_list_conf['city']==city), 'Bad_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('AfD'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['reduction'].values * 0.19
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('CDU')) & (votes_list_conf['city']==city), 'Bad_night_Die Linke'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('CDU'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party'].str.contains('Die Linke'))]['reduction'].values * 0.04

        #AfD
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any():     
            votes_list_conf.loc[(votes_list_conf['party']=='SPD') & (votes_list_conf['city']==city), 'Bad_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='SPD')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['reduction'].values * 0.03
        
        if votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any() & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('FDP').any()):     
            votes_list_conf.loc[(votes_list_conf['party']=='FDP') & (votes_list_conf['city']==city), 'Bad_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party']=='FDP')]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['reduction'].values * 0.03
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Grüne').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Grüne')) & (votes_list_conf['city']==city), 'Bad_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Grüne'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['reduction'].values * 0.03
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('CDU')) & (votes_list_conf['city']==city), 'Bad_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('CDU'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['reduction'].values * 0.0
        
        if (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('AfD').any()) & (votes_list_conf[votes_list_conf['city'] == city]['party'].str.contains('Die Linke').any()):     
            votes_list_conf.loc[(votes_list_conf['party'].str.contains('Die Linke')) & (votes_list_conf['city']==city), 'Bad_night_AfD'] = votes_list_conf[(votes_list_conf['city']==city)&(votes_list_conf['party'].str.contains('Die Linke'))]['grad_boost_pred'].values - votes_list_conf[(votes_list_conf['city']==city) & (votes_list_conf['party']=='AfD')]['reduction'].values * 0.05

votes_list_conf

C:\Users\mariu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\mariu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,party,city,grad_boost_pred,lower_confident,upper_confident,divided_by,reduction,increase,Good_night_FDP,Good_night_SPD,Good_night_CDU,Good_night_Grüne,Good_night_Die Linke,Good_night_AfD,Bad_night_FDP,Bad_night_SPD,Bad_night_CDU,Bad_night_Grüne,Bad_night_Die Linke,Bad_night_AfD
0,CDU,Babenhausen,68282.733383,36893.750457,9.600580e+04,1,31388.982926,27723.068862,61992.570618,6.559488e+04,96005.802244,63986.894448,68282.733383,68282.733383,68865.402369,72458.045835,36893.750457,63109.495337,68282.733383,68282.733383
1,FDP,Babenhausen,29141.278808,9718.979252,4.486669e+04,1,19422.299556,15725.406912,44866.685720,2.882506e+04,24982.818479,28681.010351,29141.278808,29141.278808,9718.979252,31068.346094,33849.626247,28796.396272,29141.278808,29141.278808
2,FW,Babenhausen,24618.140019,9787.626483,3.375197e+04,1,14830.513536,9133.831783,24618.140019,2.461814e+04,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019
3,Grüne,Babenhausen,33323.700806,24701.637396,4.866598e+04,1,8622.063410,15342.281910,32694.684529,2.921287e+04,31937.547363,48665.982716,33323.700806,33323.700806,35265.930761,44886.104521,41484.836367,24701.637396,33323.700806,33323.700806
4,SPD,Babenhausen,49068.273308,16950.485212,6.487918e+04,1,32117.788096,15810.902542,46709.462272,6.487918e+04,41305.814027,43391.629002,49068.273308,49068.273308,54506.517184,16950.485212,50951.612284,46998.978090,49068.273308,49068.273308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,Liberal-Konservative Reformer,Wiesbaden,100925.850610,-81230.973441,3.299421e+05,40,182156.824052,229016.227067,100925.850610,1.009259e+05,100925.850610,100925.850610,100925.850610,100925.850610,100925.850610,100925.850610,100925.850610,100925.850610,100925.850610,100925.850610
334,PARTEI,Wiesbaden,34022.872206,-40842.872508,1.099206e+05,40,74865.744714,75897.707363,34022.872206,3.402287e+04,34022.872206,34022.872206,34022.872206,34022.872206,34022.872206,34022.872206,34022.872206,34022.872206,34022.872206,34022.872206
335,SPD,Wiesbaden,521063.461413,229657.804135,1.339825e+06,40,291405.657278,818761.580743,395659.628342,1.339825e+06,177119.816499,196744.581564,425366.251391,485975.846141,593985.517850,229657.804135,522194.747092,553805.706475,510449.227625,517647.757968
336,Unabhängige Liste Wiesbaden,Wiesbaden,123659.133520,-113904.486456,4.206205e+05,40,237563.619976,296961.346478,123659.133520,1.236591e+05,123659.133520,123659.133520,123659.133520,123659.133520,123659.133520,123659.133520,123659.133520,123659.133520,123659.133520,123659.133520


In [218]:
for column in votes_list_conf.columns[8:]:
    name = column + '_divided'
    votes_list_conf[name] = votes_list_conf[column]/votes_list_conf['divided_by']

KeyError: "['Good_night_Linke_divided' 'Bad_night_Linke_divided' 'Good_night_Linke'\n 'Bad_night_Linke'] not found in axis"

In [219]:
data_city_controls = pd.read_csv(r'C:\Users\mariu\Downloads\city_controls.csv', sep=';', encoding='latin1')
data_city_controls = data_city_controls[data_city_controls['year'] == 2021]
data_city_controls = data_city_controls[['pop_total', 'city']]
data_city_controls['city'] = data_city_controls['city'].str.replace('ä', 'ae').str.replace('ö', 'oe').str.replace('ü', 'ue').str.replace('ß', 'ss')
votes_list_conf_2 = pd.merge(data_city_controls, votes_list_conf, on='city', how='right')
votes_list_conf_2['seats_total'] = 37
votes_list_conf_2.loc[(votes_list_conf_2['pop_total'] >= 25001) & (votes_list_conf_2['pop_total'] <= 50000), 'seats_total'] = 45
votes_list_conf_2.loc[(votes_list_conf_2['pop_total'] >= 50001) & (votes_list_conf_2['pop_total'] <= 100000), 'seats_total'] = 59
votes_list_conf_2.loc[(votes_list_conf_2['pop_total'] >= 100001) & (votes_list_conf_2['pop_total'] <= 250000), 'seats_total'] = 71
votes_list_conf_2.loc[(votes_list_conf_2['pop_total'] >= 250001) & (votes_list_conf_2['pop_total'] <= 500000), 'seats_total'] = 81
votes_list_conf_2.loc[(votes_list_conf_2['pop_total'] >= 500001) & (votes_list_conf_2['pop_total'] <= 1000000), 'seats_total'] = 91
votes_list_conf_2

,pop_total,city,party,grad_boost_pred,lower_confident,upper_confident,divided_by,reduction,increase,Good_night_FDP,...,Good_night_Grüne_divided,Good_night_Die Linke_divided,Good_night_AfD_divided,Bad_night_FDP_divided,Bad_night_SPD_divided,Bad_night_CDU_divided,Bad_night_Grüne_divided,Bad_night_Die Linke_divided,Bad_night_AfD_divided,seats_total
0,16940.0,Babenhausen,CDU,68282.733383,36893.750457,9.600580e+04,1,31388.982926,27723.068862,61992.570618,...,63986.894448,68282.733383,68282.733383,68865.402369,72458.045835,36893.750457,63109.495337,68282.733383,68282.733383,37
1,16940.0,Babenhausen,FDP,29141.278808,9718.979252,4.486669e+04,1,19422.299556,15725.406912,44866.685720,...,28681.010351,29141.278808,29141.278808,9718.979252,31068.346094,33849.626247,28796.396272,29141.278808,29141.278808,37
2,16940.0,Babenhausen,FW,24618.140019,9787.626483,3.375197e+04,1,14830.513536,9133.831783,24618.140019,...,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,24618.140019,37
3,16940.0,Babenhausen,Grüne,33323.700806,24701.637396,4.866598e+04,1,8622.063410,15342.281910,32694.684529,...,48665.982716,33323.700806,33323.700806,35265.930761,44886.104521,41484.836367,24701.637396,33323.700806,33323.700806,37
4,16940.0,Babenhausen,SPD,49068.273308,16950.485212,6.487918e+04,1,32117.788096,15810.902542,46709.462272,...,43391.629002,49068.273308,49068.273308,54506.517184,16950.485212,50951.612284,46998.978090,49068.273308,49068.273308,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13515,278474.0,Wiesbaden,Liberal-Konservative Reformer,100925.850610,-81230.973441,3.299421e+05,40,182156.824052,229016.227067,100925.850610,...,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,2523.146265,81
13516,278474.0,Wiesbaden,PARTEI,34022.872206,-40842.872508,1.099206e+05,40,74865.744714,75897.707363,34022.872206,...,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805,850.571805,81
13517,278474.0,Wiesbaden,SPD,521063.461413,229657.804135,1.339825e+06,40,291405.657278,818761.580743,395659.628342,...,4918.614539,10634.156285,12149.396154,14849.637946,5741.445103,13054.868677,13845.142662,12761.230691,12941.193949,81
13518,278474.0,Wiesbaden,Unabhängige Liste Wiesbaden,123659.133520,-113904.486456,4.206205e+05,40,237563.619976,296961.346478,123659.133520,...,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,3091.478338,81


In [243]:
data_conf_sorted_1 = pd.DataFrame()

#for estimator in ('rank_votes', 'rank_votes_gb', 'rank_votes_lasso', 'rank_votes_rf', 'rank_votes_vp', 'rank_votes_nn'):
    
for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Good_night_FDP_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_1 = data_conf_sorted_1.append(data_city)
data_conf_sorted_1.rename({'divided_by': 'seats_FDP_gn', 'party': 'party_FDP_gn'}, axis=1, inplace=True)
data_conf_sorted_1.reset_index(inplace=True)

data_conf_sorted_2 = pd.DataFrame()

for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Bad_night_FDP_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_2 = data_conf_sorted_2.append(data_city)
data_conf_sorted_2.rename({'divided_by': 'seats_FDP_bn', 'party': 'party_FDP_bn'}, axis=1, inplace=True)
data_conf_sorted_2.reset_index(inplace=True)

data_conf_sorted_3 = pd.DataFrame()
for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Good_night_CDU_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_3 = data_conf_sorted_3.append(data_city)
data_conf_sorted_3.rename({'divided_by': 'seats_CDU_gn', 'party': 'party_CDU_gn'}, axis=1, inplace=True)
data_conf_sorted_3.reset_index(inplace=True)

data_conf_sorted_4 = pd.DataFrame()

for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Bad_night_CDU_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_4 = data_conf_sorted_4.append(data_city)
data_conf_sorted_4.rename({'divided_by': 'seats_CDU_bn', 'party': 'party_CDU_bn'}, axis=1, inplace=True)
data_conf_sorted_4.reset_index(inplace=True)

data_conf_sorted_5 = pd.DataFrame()
for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Good_night_SPD_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_5 = data_conf_sorted_5.append(data_city)
data_conf_sorted_5.rename({'divided_by': 'seats_SPD_gn', 'party': 'party_SPD_gn'}, axis=1, inplace=True)
data_conf_sorted_5.reset_index(inplace=True)

data_conf_sorted_6 = pd.DataFrame()

for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Bad_night_SPD_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_6 = data_conf_sorted_6.append(data_city)
data_conf_sorted_6.rename({'divided_by': 'seats_SPD_bn', 'party': 'party_SPD_bn'}, axis=1, inplace=True)
data_conf_sorted_6.reset_index(inplace=True)

data_conf_sorted_7 = pd.DataFrame()
for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Good_night_Grüne_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_7 = data_conf_sorted_7.append(data_city)
data_conf_sorted_7.rename({'divided_by': 'seats_Grüne_gn', 'party': 'party_Grüne_gn'}, axis=1, inplace=True)
data_conf_sorted_7.reset_index(inplace=True)

data_conf_sorted_8 = pd.DataFrame()

for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Bad_night_Grüne_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_8 = data_conf_sorted_8.append(data_city)
data_conf_sorted_8.rename({'divided_by': 'seats_Grüne_bn', 'party': 'party_Grüne_bn'}, axis=1, inplace=True)
data_conf_sorted_8.reset_index(inplace=True)

data_conf_sorted_9 = pd.DataFrame()
for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Good_night_AfD_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_9 = data_conf_sorted_9.append(data_city)
data_conf_sorted_9.rename({'divided_by': 'seats_AfD_gn', 'party': 'party_AfD_gn'}, axis=1, inplace=True)
data_conf_sorted_9.reset_index(inplace=True)

data_conf_sorted_10 = pd.DataFrame()

for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Bad_night_AfD_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_10 = data_conf_sorted_10.append(data_city)
data_conf_sorted_10.rename({'divided_by': 'seats_AfD_bn', 'party': 'party_AfD_bn'}, axis=1, inplace=True)
data_conf_sorted_10.reset_index(inplace=True)

data_conf_sorted_11 = pd.DataFrame()

for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Good_night_Die Linke_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_11 = data_conf_sorted_11.append(data_city)
data_conf_sorted_11.rename({'divided_by': 'seats_Die Linke_gn', 'party': 'party_Die Linke_gn'}, axis=1, inplace=True)
data_conf_sorted_11.reset_index(inplace=True)

data_conf_sorted_12 = pd.DataFrame()

for city in votes_list_conf_2['city'].unique():
        
    data_city = votes_list_conf_2[votes_list_conf_2['city'] == city]
    data_city = data_city.sort_values(by='Bad_night_Die Linke_divided', ascending=False).reset_index(drop=True)
    nseats = data_city['seats_total'].values - 1
    data_city = data_city[data_city.index <= nseats][['city', 'party', 'divided_by', 'seats_total']]
    
    data_conf_sorted_12 = data_conf_sorted_12.append(data_city)
data_conf_sorted_12.rename({'divided_by': 'seats_Die Linke_bn', 'party': 'party_Die Linke_bn'}, axis=1, inplace=True)
data_conf_sorted_12.reset_index(inplace=True)

In [248]:
data_conf_sorted = data_conf_sorted_1.join(data_conf_sorted_2[['party_FDP_bn', 'seats_FDP_bn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_3[['party_CDU_gn', 'seats_CDU_gn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_4[['party_CDU_bn', 'seats_CDU_bn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_5[['party_SPD_gn', 'seats_SPD_gn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_6[['party_SPD_bn', 'seats_SPD_bn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_7[['party_Grüne_gn', 'seats_Grüne_gn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_8[['party_Grüne_bn', 'seats_Grüne_bn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_9[['party_AfD_gn', 'seats_AfD_gn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_10[['party_AfD_bn', 'seats_AfD_bn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_11[['party_Die Linke_gn', 'seats_Die Linke_gn']])
data_conf_sorted = data_conf_sorted.join(data_conf_sorted_12[['party_Die Linke_bn', 'seats_Die Linke_bn']])
data_conf_sorted

,index,city,party_FDP_gn,seats_FDP_gn,seats_total,party_FDP_bn,seats_FDP_bn,party_CDU_gn,seats_CDU_gn,party_CDU_bn,...,party_Grüne_bn,seats_Grüne_bn,party_AfD_gn,seats_AfD_gn,party_AfD_bn,seats_AfD_bn,party_Die Linke_gn,seats_Die Linke_gn,party_Die Linke_bn,seats_Die Linke_bn
0,0,Babenhausen,CDU,1,37,CDU,1,CDU,1,SPD,...,CDU,1,CDU,1,CDU,1,CDU,1,CDU,1
1,1,Babenhausen,SPD,1,37,SPD,1,CDU,2,Grüne,...,SPD,1,SPD,1,SPD,1,SPD,1,SPD,1
2,2,Babenhausen,FDP,1,37,Grüne,1,SPD,1,CDU,...,CDU,2,CDU,2,CDU,2,CDU,2,CDU,2
3,3,Babenhausen,Grüne,1,37,CDU,2,CDU,3,FDP,...,FDP,1,Grüne,1,Grüne,1,Grüne,1,Grüne,1
4,4,Babenhausen,CDU,2,37,SPD,2,Grüne,1,SPD,...,Grüne,1,FDP,1,FDP,1,FDP,1,FDP,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2171,76,Wiesbaden,FDP,33,81,SPD,21,Unabhängige Liste Wiesbaden,4,SPD,...,AfD,4,AfD,13,Grüne,13,Unabhängige Liste Wiesbaden,4,SPD,20
2172,77,Wiesbaden,Bürgerliste Wiesbaden,3,81,SPD,22,SPD,6,CDU,...,Bürgerliste Wiesbaden,3,SPD,16,CDU,19,SPD,14,Liberal-Konservative Reformer,4
2173,78,Wiesbaden,FW,3,81,CDU,19,Grüne,10,SPD,...,FW,3,CDU,14,FDP,9,CDU,16,CDU,20
2174,79,Wiesbaden,FDP,34,81,Grüne,14,FDP,2,Grüne,...,Unabhängige Liste Wiesbaden,4,Grüne,12,SPD,20,AfD,4,Unabhängige Liste Wiesbaden,5


In [250]:
data_conf_sorted.to_csv(r'C:\Users\mariu\Documents\hessen_conf_2.csv')